In [1]:
import fatf
import fatf.utils.data.datasets as fatf_datasets
from scipy import stats 
import sklearn.ensemble
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D 
# %matplotlib notebook
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
import matplotlib
import matplotlib.pyplot as plt
import operator as op
from functools import reduce

import time
import warnings

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

In [2]:
from sklearn.utils import check_random_state

In [3]:
from matplotlib.patches import Ellipse
import math
from matplotlib.patches import Rectangle
from matplotlib.patches import Ellipse

In [4]:
from lime.discretize import BaseDiscretizer
class GaussianMixtureModelsDiscretizer(BaseDiscretizer):
    
    def __init__(self, train, categorical_features, feature_names, labels, random_state=None, data_stats=None):
        self.train = train
        self.labels = labels
        
        if(labels is None):
            raise ValueError('Labels must be not None when using \
                             Gaussian Mixture Models Discretizer')
            
        BaseDiscretizer.__init__(self, train, categorical_features,
                                 feature_names, labels=labels,
                                 random_state=random_state,
                                 data_stats=data_stats)

    
    def getLabels(self):
        train_label = []
        for lebel_number in set(self.labels):
            Classification = np.array([self.train[i] for i in range(len(self.labels)) if self.labels[i] == lebel_number])
            train_label.append(Classification)
        
        return train_label
    
    # BIC_range = make it an input??
    def gmmBIC(self, train_label_list, BIC_range = [1, 10]):
    
        all_gmm_list = []
        for class_C in train_label_list:
            gmm_list = []
            number_features = len(class_C[0])
            for i in range(number_features): 
                BIC_list = []
                for x in range(min(BIC_range), max(BIC_range) + 1):
                    gmm = mixture.GaussianMixture(n_components= x, covariance_type='full').fit(class_C[:, i].reshape(-1, 1))
                    BIC_list.append(gmm.bic(class_C[:, i].reshape(-1, 1)))

                min_BIC_index = BIC_list.index(min(BIC_list))
                number_components = min_BIC_index + 1
                gmm = mixture.GaussianMixture(n_components=number_components, covariance_type='full').fit(class_C[:, i].reshape(-1, 1))
                gmm_list.append(gmm)

            all_gmm_list.append(gmm_list)

        return all_gmm_list
    
    def getIntervalsGMM(self, gmm, std_constant=3):
    
        width_interval_list = []
        pos_list = []
        weight_list = []
        for pos, covariance, weight in zip(gmm.means_, gmm.covariances_, gmm.weights_):

            width = [float(pos -std_constant * np.sqrt(covariance[0][0])), float(pos + std_constant * np.sqrt(covariance[0][0]))]

            width_interval_list.append(width)
            pos_list.append(pos)
            weight_list.append(weight)


        return pos_list, width_interval_list, weight_list
    
    
    def discretizeGMM(self, gmm_all, std_constant=3):
        
        n_classes = len(gmm_all)
        n_features = len(gmm_all[0])

        features_discretized = []
        for n in range(n_features):
            inteval_features = []
            for i in range(n_classes):
                _, intevals_list, _  = self.getIntervalsGMM(gmm_all[i][n], std_constant)
                flat_intervals = sorted([interval for intervals in intevals_list for interval in intervals])
                inteval_features.append(flat_intervals)

            inteval_features_flat = sorted([interval for intervals in inteval_features for interval in intervals])
            features_discretized.append(np.array(inteval_features_flat))


        return features_discretized

    
      
        
    def bins(self, data, labels):
        
        train_label = self.getLabels()
        all_gmm_list = self.gmmBIC(train_label)
        feature_intervals_lists = self.discretizeGMM(all_gmm_list)
        bins = feature_intervals_lists
        
        return bins
    
    

In [58]:
adult_path = r'Adult_data_set\adult_data.txt'
df = pd.read_csv(adult_path, header=None)

adult_path_test = r'Adult_data_set\adult_test.txt'
df_test = pd.read_csv(adult_path_test, header=None)

In [59]:
y_d = pd.get_dummies(df.iloc[:, -1])
y = y_d[' >50K']

y_d_test = pd.get_dummies(df_test.iloc[:, -1])
y_test = y_d_test[' >50K.']

In [60]:
def Numerice(df_column):
    unique = df_column.unique()
    status_unique = dict(zip(unique, range(len(unique))))
    df_column = df_column.replace(status_unique)
    return df_column

In [61]:
n_columns = len(df.columns)

for i in range(n_columns):
    df[i]= Numerice(df[i])
    df_test[i]= Numerice(df_test[i])

In [68]:
X = df.iloc[:, :8]
X_test = df_test.iloc[:, :8]

In [69]:
target_names = list(y_d.columns)
feature_names = list(X.columns)
len(feature_names)

8

In [70]:
X = np.array(X)
y = np.array(y)

In [71]:
X 

array([[    0,     0,     0, ...,     0,     0,     0],
       [    1,     1,     1, ...,     1,     1,     1],
       [    2,     2,     2, ...,     2,     2,     0],
       ...,
       [   50,     2, 19507, ...,     6,     0,     4],
       [   24,     2, 17888, ...,     0,     0,     3],
       [    7,     6, 14492, ...,     1,     1,     2]], dtype=int64)

In [72]:
list_feature_names = [str(i) for i in feature_names]
feature_names = list_feature_names
print(len(feature_names))

8


In [73]:
blackbox_model = sklearn.ensemble.RandomForestClassifier(n_estimators=20)
blackbox_model.fit(X, y)
print('Train', sklearn.metrics.accuracy_score(y, blackbox_model.predict(X)))
print('Test', sklearn.metrics.accuracy_score(y_test, blackbox_model.predict(X_test)))

Train 0.994256933140874
Test 0.7371168847122412


In [74]:
import lime
import lime.lime_tabular
import collections
import sklearn
import numpy as np
import os
import copy
import string
from io import open
import json
import matplotlib

In [75]:
"""  anchor_base.py 
     https://github.com/marcotcr/anchor/blob/master/anchor/anchor_base.py  """ 
from __future__ import print_function
# import numpy as np
import operator
import copy
# import sklearn
# import collections

""" Used in def get_anchor_from_tuple"""
def matrix_subset(matrix, n_samples):
    if matrix.shape[0] == 0:
        return matrix
    n_samples = min(matrix.shape[0], n_samples)
    return matrix[np.random.choice(matrix.shape[0], n_samples, replace=False)]

""" ~~~~ Used in calss AnchorTabularExplainer ~~~~ """
class AnchorBaseBeam(object):
    def __init__(self):
        pass
    
    """ KL divergence between two multivariate Bernoulli """
    
    """ Used in dlow_bernoulli """
    """The log likelihood ratio can be interpreted as the amount of evidence the data provide for one model versus another, 
        so the KL divergence tells us how much evidence we can expect our data to provide in favor of the true model."""
    
    """ D(P||D) = It a measure of how one probability distribution (P) is different from a second (Q)"""
    """ It is also called relative entropy """
    """ It is not the distance between two distribution- often misundestood """
    """ p and q are the probability distributions """
    """ p to q !=  q to p -> nonsymmetric """
    """ D(P||D) = is the information gain when distribution Q is used insted of distibution P """
    @staticmethod
    def kl_bernoulli(p, q):
        p = min(0.9999999999999999, max(0.0000001, p))
        q = min(0.9999999999999999, max(0.0000001, q))
        return (p * np.log(float(p) / q) + (1 - p) *
                np.log(float(1 - p) / (1 - q)))
    
    """ Used in anchor_beam """
    @staticmethod
    def compute_beta(n_features, t, delta):
        alpha = 1.1
        k = 405.5
        temp = np.log(k * n_features * (t ** alpha) / delta)
        return temp + np.log(temp)
    
    """ Used in lucb and anchor_beam """
    @staticmethod
    def dup_bernoulli(p, level):
        lm = p
        um = min(min(1, p + np.sqrt(level / 2.)), 1)
        qm = (um + lm) / 2.
#         print 'lm', lm, 'qm', qm, kl_bernoulli(p, qm)
        if AnchorBaseBeam.kl_bernoulli(p, qm) > level:
            um = qm
        else:
            lm = qm
        return um
    """ Used in anchor_beam """
    @staticmethod
    def lucb(sample_fns, initial_stats, epsilon, delta, batch_size, top_n,
             verbose=False, verbose_every=1):
        # initial_stats must have n_samples, positive
        n_features = len(sample_fns)
        n_samples = np.array(initial_stats['n_samples'])
        positives = np.array(initial_stats['positives'])
        ub = np.zeros(n_samples.shape)
        lb = np.zeros(n_samples.shape)
        for f in np.where(n_samples == 0)[0]:
            n_samples[f] += 1
            positives[f] += sample_fns[f](1)
        if n_features == top_n:
            return range(n_features)
        means = positives / n_samples
        t = 1

        def update_bounds(t):
            sorted_means = np.argsort(means)
            beta = AnchorBaseBeam.compute_beta(n_features, t, delta)
            J = sorted_means[-top_n:]
            not_J = sorted_means[:-top_n]
            for f in not_J:
                ub[f] = AnchorBaseBeam.dup_bernoulli(means[f], beta /
                                                     n_samples[f])
            for f in J:
                lb[f] = AnchorBaseBeam.dlow_bernoulli(means[f],
                                                      beta / n_samples[f])
            ut = not_J[np.argmax(ub[not_J])]
            lt = J[np.argmin(lb[J])]
            return ut, lt
        ut, lt = update_bounds(t)
        B = ub[ut] - lb[lt]
        verbose_count = 0
        while B > epsilon:
            verbose_count += 1
            if verbose and verbose_count % verbose_every == 0:
                print('Best: %d (mean:%.10f, n: %d, lb:%.4f)' %
                      (lt, means[lt], n_samples[lt], lb[lt]), end=' ')
                print('Worst: %d (mean:%.4f, n: %d, ub:%.4f)' %
                      (ut, means[ut], n_samples[ut], ub[ut]), end=' ')
                print('B = %.2f' % B)
            n_samples[ut] += batch_size
            positives[ut] += sample_fns[ut](batch_size)
            means[ut] = positives[ut] / n_samples[ut]
            n_samples[lt] += batch_size
            positives[lt] += sample_fns[lt](batch_size)
            means[lt] = positives[lt] / n_samples[lt]
            t += 1
            ut, lt = update_bounds(t)
            B = ub[ut] - lb[lt]
        sorted_means = np.argsort(means)
        return sorted_means[-top_n:]
    
    """ Used in anchor_beam """
    @staticmethod
    def dlow_bernoulli(p, level):
        um = p
        lm = max(min(1, p - np.sqrt(level / 2.)), 0)
        qm = (um + lm) / 2.
#         print 'lm', lm, 'qm', qm, kl_bernoulli(p, qm)
        if AnchorBaseBeam.kl_bernoulli(p, qm) > level:
            lm = qm
        
        else:
            um = qm
        return lm

    """ Used in anchor_beam """
    @staticmethod
    def make_tuples(previous_best, state):
        # alters state, computes support for new tuples
        normalize_tuple = lambda x: tuple(sorted(set(x)))  # noqa
        all_features = range(state['n_features'])
        coverage_data = state['coverage_data']
        current_idx = state['current_idx']
        data = state['data'][:current_idx]
        labels = state['labels'][:current_idx]
        
        """ len(previous_best) == 0 ->  first try"""
        if len(previous_best) == 0:
            tuples = [(x, ) for x in all_features]
#             print("all_features", all_features)
#             print("tuples", tuples)
            for x in tuples:
                pres = data[:, x[0]].nonzero()[0]
#                 print("pres", pres)
#                 print("float(len(pres)", float(len(pres)))
#                 print("float(labels[pres].sum())", float(labels[pres].sum()))
#                 print("x[0]", x[0])
                # NEW
                state['t_idx'][x] = set(pres)
                state['t_nsamples'][x] = float(len(pres))
                state['t_positives'][x] = float(labels[pres].sum())
                state['t_order'][x].append(x[0])
                # NEW
                state['t_coverage_idx'][x] = set(
                    coverage_data[:, x[0]].nonzero()[0])
                state['t_coverage'][x] = (
                    float(len(state['t_coverage_idx'][x])) /
                    coverage_data.shape[0])
            return tuples
#         print("")
        new_tuples = set()
        for f in all_features:
            for t in previous_best:
                new_t = normalize_tuple(t + (f, ))
                if len(new_t) != len(t) + 1:
                    continue
                if new_t not in new_tuples:
                    new_tuples.add(new_t)
                    state['t_order'][new_t] = copy.deepcopy(state['t_order'][t])
                    state['t_order'][new_t].append(f)
                    state['t_coverage_idx'][new_t] = (
                        state['t_coverage_idx'][t].intersection(
                            state['t_coverage_idx'][(f,)]))
                    state['t_coverage'][new_t] = (
                        float(len(state['t_coverage_idx'][new_t])) /
                        coverage_data.shape[0])
                    t_idx = np.array(list(state['t_idx'][t]))
                    t_data = state['data'][t_idx]
                    present = np.where(t_data[:, f] == 1)[0]
                    state['t_idx'][new_t] = set(t_idx[present])
                    idx_list = list(state['t_idx'][new_t])
                    state['t_nsamples'][new_t] = float(len(idx_list))
                    state['t_positives'][new_t] = np.sum(
                        state['labels'][idx_list])
        return list(new_tuples)
    
    """ Used in anchor_beam """
    @staticmethod
    def get_sample_fns(sample_fn, tuples, state):
        # each sample fn returns number of positives
        sample_fns = []
        def complete_sample_fn(t, n):
            raw_data, data, labels = sample_fn(list(t), n)
            current_idx = state['current_idx']
            # idxs = range(state['data'].shape[0], state['data'].shape[0] + n)
            idxs = range(current_idx, current_idx + n)
            state['t_idx'][t].update(idxs)
            state['t_nsamples'][t] += n
            state['t_positives'][t] += labels.sum()
            state['data'][idxs] = data
            state['raw_data'][idxs] = raw_data
            state['labels'][idxs] = labels
            state['current_idx'] += n
            if state['current_idx'] >= state['data'].shape[0] - max(1000, n):
                prealloc_size = state['prealloc_size']
                current_idx = data.shape[0]
                state['data'] = np.vstack(
                    (state['data'],
                     np.zeros((prealloc_size, data.shape[1]), data.dtype)))
                state['raw_data'] = np.vstack(
                    (state['raw_data'],
                     np.zeros((prealloc_size, raw_data.shape[1]),
                              raw_data.dtype)))
                state['labels'] = np.hstack(
                    (state['labels'],
                     np.zeros(prealloc_size, labels.dtype)))
            # This can be really slow
            # state['data'] = np.vstack((state['data'], data))
            # state['raw_data'] = np.vstack((state['raw_data'], raw_data))
            # state['labels'] = np.hstack((state['labels'], labels))
            return labels.sum()
        for t in tuples:
            sample_fns.append(lambda n, t=t: complete_sample_fn(t, n))
        
#         print("sample_fns", sample_fns)
        return sample_fns
    
    """ Used in anchor_beam """
    @staticmethod
    def get_initial_statistics(tuples, state):
        stats = {
            'n_samples': [],
            'positives': []
        }
        for t in tuples:
            stats['n_samples'].append(state['t_nsamples'][t])
            stats['positives'].append(state['t_positives'][t])
        return stats
    
    """ Used in anchor_beam """
    @staticmethod
    def get_anchor_from_tuple(t, state):
        # TODO: This is wrong, some of the intermediate anchors may not exist.
        anchor = {'feature': [], 'mean': [], 'precision': [],
                  'coverage': [], 'examples': [], 'all_precision': 0}
        anchor['num_preds'] = state['data'].shape[0]
        normalize_tuple = lambda x: tuple(sorted(set(x)))  # noqa
        current_t = tuple()
        for f in state['t_order'][t]:
            current_t = normalize_tuple(current_t + (f,))

            mean = (state['t_positives'][current_t] /
                    state['t_nsamples'][current_t])
            anchor['feature'].append(f)
            anchor['mean'].append(mean)
            anchor['precision'].append(mean)
            anchor['coverage'].append(state['t_coverage'][current_t])
            raw_idx = list(state['t_idx'][current_t])
            raw_data = state['raw_data'][raw_idx]
            covered_true = (
                state['raw_data'][raw_idx][state['labels'][raw_idx] == 1])
            covered_false = (
                state['raw_data'][raw_idx][state['labels'][raw_idx] == 0])
            exs = {}
            exs['covered'] = matrix_subset(raw_data, 10)
            exs['covered_true'] = matrix_subset(covered_true, 10)
            exs['covered_false'] = matrix_subset(covered_false, 10)
            exs['uncovered_true'] = np.array([])
            exs['uncovered_false'] = np.array([])
            anchor['examples'].append(exs)
        return anchor
    
    """ I think is where the best anchor is selected (using mult)"""
    @staticmethod
    def anchor_beam(sample_fn, delta=0.05, epsilon=0.1, batch_size=10,
                    min_shared_samples=0, desired_confidence=1, beam_size=1,
                    verbose=False, epsilon_stop=0.05, min_samples_start=0,
                    max_anchor_size=None, verbose_every=1,
                    stop_on_first=False, coverage_samples=10000):
        anchor = {'feature': [], 'mean': [], 'precision': [],
                  'coverage': [], 'examples': [], 'all_precision': 0}
        
        """ sample_fn returns: raw_data, data, labels
                raw_data = boostram samples. 
                data = array of 1 and 0, 1 if the train_dataset is <=, > or == to the rigion... 
                labels = array of 1 and 0, 1 if the predictions is the same as the true labels, 0 otherwise """
        """coverage_data =  array of 1000 list with 12 elements, each with 0 or 1"""
        _, coverage_data, _ = sample_fn([], coverage_samples, compute_labels=False)
        
#         print("coverage_data", coverage_data)
#         print("len(coverage_data)", len(coverage_data))
#         print("len(coverage_data[0])", len(coverage_data[0]))
        
        """max(1, min_samples_start) = max(1, 0) = 1"""
        """instance = [] """
        """ raw_data = one random boostrap example, data=[1 0 ... 0], labels = predictions"""
        raw_data, data, labels = sample_fn([], max(1, min_samples_start))
        
#         print("raw_data, data, labels", raw_data, data, labels)
        
        mean = labels.mean()
        beta = np.log(1. / delta)
#         print("beta / data.shape[0]", beta / data.shape[0])
        lb = AnchorBaseBeam.dlow_bernoulli(mean, beta / data.shape[0])
        
        """ desired_confidence = 1, epsilon = 0.1 """
        while mean > desired_confidence and lb < desired_confidence - epsilon:
            """batch_size=10 """
            """ nraw_data = 10 random boostrap example, ndata=[1 0 ... 0], nlabels = predictions"""
            nraw_data, ndata, nlabels = sample_fn([], batch_size)
#             print("nraw_data, ndata, nlabels", nraw_data, ndata, nlabels)
            """ stacks data, row_data and labels, I don't really undestand the point of this"""
            data = np.vstack((data, ndata))
            raw_data = np.vstack((raw_data, nraw_data))
            raw_data = np.vstack((raw_data, nraw_data))
            labels = np.hstack((labels, nlabels))
#             print("raw_data, data, labels", raw_data, data, labels)
            mean = labels.mean()
            lb = AnchorBaseBeam.dlow_bernoulli(mean, beta / data.shape[0])
        if lb > desired_confidence:
            anchor['num_preds'] = data.shape[0]
            anchor['all_precision'] = mean
            print("if lb > desired_confidence", lb > desired_confidence)
            print("anchor['num_preds']", anchor['num_preds'])
            print("anchor['all_precision']", anchor['all_precision'])
            return anchor
        
        """ batch_size = 10 -> prealloc_size = 100000 """
        prealloc_size = batch_size * 10000
        current_idx = data.shape[0]
        data = np.vstack((data, np.zeros((prealloc_size, data.shape[1]),
                                         data.dtype)))
        raw_data = np.vstack(
            (raw_data, np.zeros((prealloc_size, raw_data.shape[1]),
                                raw_data.dtype)))
        labels = np.hstack((labels, np.zeros(prealloc_size, labels.dtype)))
        n_features = data.shape[1]
        
#         print("prealloc_size", prealloc_size)
#         print("current_idx", current_idx)
#         print("data", data)
#         print("raw_data", raw_data)
#         print("labels", labels)
#         print("n_features", n_features)
        state = {'t_idx': collections.defaultdict(lambda: set()),
                 't_nsamples': collections.defaultdict(lambda: 0.),
                 't_positives': collections.defaultdict(lambda: 0.),
                 'data': data,
                 'prealloc_size': prealloc_size,
                 'raw_data': raw_data,
                 'labels': labels,
                 'current_idx': current_idx,
                 'n_features': n_features,
                 't_coverage_idx': collections.defaultdict(lambda: set()),
                 't_coverage': collections.defaultdict(lambda: 0.),
                 'coverage_data': coverage_data,
                 't_order': collections.defaultdict(lambda: list())
                 }
        current_size = 1
        best_of_size = {0: []}
        best_coverage = -1
        best_tuple = ()
        t = 1
        """ max_anchor_size = None, n_features= 12 (most of the times)"""
        if max_anchor_size is None:
            max_anchor_size = n_features
            
        """ current_size = 1 """
        while current_size <= max_anchor_size:
            """ makes tuples from all_features """
            tuples = AnchorBaseBeam.make_tuples(
                best_of_size[current_size - 1], state)
#             print("\n tuples before", tuples)
            tuples = [x for x in tuples
                      if state['t_coverage'][x] > best_coverage]
#             print("best_coverage", best_coverage)
#             print("tuples after", tuples, "\n")
            
            """ len(tuples) == 0 -> Non of new tubles have a lager coverage compared to the best one found preciously """
            if len(tuples) == 0:
                break
            
            sample_fns = AnchorBaseBeam.get_sample_fns(sample_fn, tuples,
                                                       state)
#             print("sample_fns", sample_fns)
            initial_stats = AnchorBaseBeam.get_initial_statistics(tuples, state)
#             print("initial_stats", initial_stats)
            
            # print tuples, beam_size
            
            """ still yet to undestand how this function works """
            chosen_tuples = AnchorBaseBeam.lucb(
                sample_fns, initial_stats, epsilon, delta, batch_size,
                min(beam_size, len(tuples)),
                verbose=verbose, verbose_every=verbose_every)
            
            best_of_size[current_size] = [tuples[x] for x in chosen_tuples]
            
            """ Verbose =  False, Verbose that does this mean? """
            if verbose:
                print('Best of size ', current_size, ':')
            # print state['data'].shape[0]
            stop_this = False
            for i, t in zip(chosen_tuples, best_of_size[current_size]):
                # I can choose at most (beam_size - 1) tuples at each step,
                # and there are at most n_feature steps
                """ why does it chose this beta? """
                beta = np.log(1. /
                              (delta / (1 + (beam_size - 1) * n_features)))
                # beta = np.log(1. / delta)
                # if state['t_nsamples'][t] == 0:
                #     mean = 1
                # else:
                mean = state['t_positives'][t] / state['t_nsamples'][t]
                lb = AnchorBaseBeam.dlow_bernoulli(
                    mean, beta / state['t_nsamples'][t])
                ub = AnchorBaseBeam.dup_bernoulli(
                    mean, beta / state['t_nsamples'][t])
                coverage = state['t_coverage'][t]
                if verbose:
                    print(i, mean, lb, ub)
                while ((mean >= desired_confidence and
                       lb < desired_confidence - epsilon_stop) or
                       (mean < desired_confidence and
                        ub >= desired_confidence + epsilon_stop)):
                    # print mean, lb, state['t_nsamples'][t]
                    sample_fns[i](batch_size)
                    mean = state['t_positives'][t] / state['t_nsamples'][t]
                    lb = AnchorBaseBeam.dlow_bernoulli(
                        mean, beta / state['t_nsamples'][t])
                    ub = AnchorBaseBeam.dup_bernoulli(
                        mean, beta / state['t_nsamples'][t])
                if verbose:
                    print('%s mean = %.2f lb = %.2f ub = %.2f coverage: %.2f n: %d' % (t, mean, lb, ub, coverage, state['t_nsamples'][t]))
                if mean >= desired_confidence and lb > desired_confidence - epsilon_stop:
                    if verbose:
                        print('Found eligible anchor ', t, 'Coverage:',
                              coverage, 'Is best?', coverage > best_coverage)
                    if coverage > best_coverage:
                        best_coverage = coverage
                        best_tuple = t
                        if best_coverage == 1 or stop_on_first:
                            stop_this = True
            if stop_this:
                break
            current_size += 1
        if best_tuple == ():
            # Could not find an anchor, will now choose the highest precision
            # amongst the top K from every round
            if verbose:
                print('Could not find an anchor, now doing best of each size')
            tuples = []
            for i in range(0, current_size):
                tuples.extend(best_of_size[i])
            # tuples = best_of_size[current_size - 1]
            sample_fns = AnchorBaseBeam.get_sample_fns(sample_fn, tuples,
                                                       state)
            initial_stats = AnchorBaseBeam.get_initial_statistics(tuples,
                                                                  state)
            # print tuples, beam_size
            chosen_tuples = AnchorBaseBeam.lucb(
                sample_fns, initial_stats, epsilon, delta, batch_size,
                1, verbose=verbose)
            best_tuple = tuples[chosen_tuples[0]]
        # return best_tuple, state
        return AnchorBaseBeam.get_anchor_from_tuple(best_tuple, state)

In [76]:
from __future__ import print_function
import io


""" ~~~~~ Used in class AnchorBaseBeam(object): ~~~~~ """
class AnchorExplanation:
    """Object returned by explainers"""
    def __init__(self, type_, exp_map, as_html):
        self.type = type_
        self.exp_map = exp_map
        self.as_html_fn = as_html
    
    """ Not in the main code, just when printting the rages of the anchor's features"""
    def names(self, partial_index=None):
        """
        Returns a list of the names of the anchor conditions.
        Args:
            partial_index (int): lets you get the anchor until a certain index.
            For example, if the anchor is (A=1,B=2,C=2) and partial_index=1,
            this will return ["A=1", "B=2"]
        """
        names = self.exp_map['names']
        if partial_index is not None:
            names = names[:partial_index + 1]
        return names
    
    """ Not in the main code, returns the index of the anchor's features used"""
    def features(self, partial_index=None):
        """
        Returns a list of the features used in the anchor conditions.
        Args:
            partial_index (int): lets you get the anchor until a certain index.
            For example, if the anchor uses features (1, 2, 3) and
            partial_index=1, this will return [1, 2]
        """
        features = self.exp_map['feature']
        if partial_index is not None:
            features = features[:partial_index + 1]
        return features
    
    """ Not in the main code, returns precision of the anchor's"""
    def precision(self, partial_index=None):
        """
        Returns the anchor precision (a float)
        Args:
            partial_index (int): lets you get the anchor precision until a
            certain index. For example, if the anchor has precisions
            [0.1, 0.5, 0.95] and partial_index=1, this will return 0.5
        """
        precision = self.exp_map['precision']
        if len(precision) == 0:
            return self.exp_map['all_precision']
        if partial_index is not None:
            return precision[partial_index]
        else:
            return precision[-1]

    """ Not in the main code, returns coverage of the anchor's """
    def coverage(self, partial_index=None):
        """
        Returns the anchor coverage (a float)
        Args:
            partial_index (int): lets you get the anchor coverage until a
            certain index. For example, if the anchor has coverages
            [0.1, 0.5, 0.95] and partial_index=1, this will return 0.5
        """
        coverage = self.exp_map['coverage']
        if len(coverage) == 0:
            return 1
        if partial_index is not None:
            return coverage[partial_index]
        else:
            return coverage[-1]

In [77]:
""" anchor_tabular.py 
    https://github.com/marcotcr/anchor/blob/master/anchor/anchor_tabular.py"""

"""def as_html(, def jsonize(x)"""

def id_generator(size=15):
    """Helper function to generate random div ids. This is useful for embedding
    HTML into ipython notebooks."""
    chars = list(string.ascii_uppercase + string.digits)

    
    
""" This is the main class-file to compute the anchors for tabular data """
class AnchorTabularExplainer(object):
    """
        Args:
            class_names: list of strings
            feature_names: list of strings
            train_data: used to sample (bootstrap)
            categorical_names: map from integer to list of strings, names for each
                value of the categorical features. Every feature that is not in
                this map will be considered as ordinal or continuous, and thus discretized.
        """

    def __init__(self, class_names, feature_names, train_data, train_label = None,
                 categorical_names={}, discretizer='quartile', encoder_fn=None):
        self.min = {}
        self.max = {}
        self.disc = collections.namedtuple('random_name2',
                                              ['discretize'])(lambda x: x)
        self.encoder_fn = lambda x: x
        if encoder_fn is not None:
            self.encoder_fn = encoder_fn
        self.categorical_features = []
        self.feature_names = feature_names
        self.train = train_data
        self.class_names = class_names
        self.categorical_names = copy.deepcopy(categorical_names)
        if categorical_names:
            self.categorical_features = sorted(categorical_names.keys())

        if discretizer == 'quartile':
            self.disc = QuartileDiscretizer(train_data,self.categorical_features,
                                                         self.feature_names)
        elif discretizer == 'decile':
            self.disc = lime.lime_tabular.DecileDiscretizer(train_data,
                                                     self.categorical_features,
                                                     self.feature_names)
        elif discretizer == "entropy":
            self.disc = lime.lime_tabular.EntropyDiscretizer(train_data,
                                                     self.categorical_features,
                                                     self.feature_names, train_label)
            
        elif discretizer == 'GaussianMixtureModels':
            self.disc = GaussianMixtureModelsDiscretizer(train_data, self.categorical_features,
                                                         self.feature_names, train_label)
            
        else:
            raise ValueError('Discretizer must be quartile, decile or GaussianMixtureModels')

        """ ?? """
        self.ordinal_features = [x for x in range(len(feature_names)) if x not in self.categorical_features]
        
#         print("self.train", self.train)
#         self.d_train = self.disc.discretize(self.train)
        self.d_train = self.disc.discretize(self.train)
#         print("self.d_train", self.d_train)
#         print("self.d_train.type()", type(self.d_train))
        self.categorical_names.update(self.disc.names)
#         print("self.categorical_names", self.categorical_names)
        self.categorical_features += self.ordinal_features
#         print("self.categorical_features", self.categorical_features)

        """ selects the min and max in the training data set"""
        for f in range(train_data.shape[1]):
            self.min[f] = np.min(train_data[:, f])
            self.max[f] = np.max(train_data[:, f])
            
    
    """ This is def explain_instance: def sample_fn """
    """Randomelly selects with replacement from the training dataset where feature is > the saved reagio
                     and samples as many samples from the boostrap <=  to the saved reagion """
    def sample_from_train(self, conditions_eq, conditions_neq, conditions_geq,
                          conditions_leq, num_samples):
        """
        bla
        """
        """ Train the list of training input dataset, d_train is the discretized training input datset."""
        train = self.train
        d_train = self.d_train
        
        """train.shape[0]=len(train)=120, , with replacement --> boostrap, 
            selects n (= num_samples) indexes from the train dataset with replacement"""
        idx = np.random.choice(range(train.shape[0]), num_samples, replace=True)
        
        """ selectes multiple samples with replacement """
        sample = train[idx] # sample = the selected train from the selected index
        d_sample = d_train[idx] # d_sample = sample discretized

        
        for f in conditions_eq:
            sample[:, f] = np.repeat(conditions_eq[f], num_samples)
        

        
        """conditions_geq[f] = the rigion of the saved from the quartile with the '<='' or ''>',
            f is the location saved in conditions_geq"""
        for f in conditions_geq:
#             print("f", f)
#             print("conditions_geq", conditions_geq)
#             print("conditions_geq[f]", conditions_geq[f])
#             print("d_sample[:, f]", d_sample[:, f])
            """ d_sample[:, f] = all of the discretized sample in the f location (index)."""
            """ idx = for all the d_sample checks in the f index is <= the saved region (conditions_geq[f]),
                returning Ture or False. """
            idx = d_sample[:, f] <= conditions_geq[f]
#             print('len(idx)', len(idx))
#             print('len(d_sample[:, f])', len(d_sample[:, f]))
#             print("\nif f in conditions_leq", f in conditions_leq)
            
            if f in conditions_leq:
                idx = (idx + (d_sample[:, f] > conditions_leq[f])).astype(bool)
            
            """ Cheeks if there all False """
            """ continue... What is the point of this?"""
            if idx.sum() == 0:
                continue
            
#             print("\nf", f)
#             print("d_train", d_train)
#             print("d_train[:, f]", d_train[:, f])
#             print("conditions_geq", conditions_geq)
#             print("conditions_geq[f]", conditions_geq[f])
            """ options  =  for all the d_train checks in the f index is > the saved region (conditions_geq[f]),
                returning Ture or False. Oposite to idex"""
            options = d_train[:, f] > conditions_geq[f]
#             print("float(conditions_geq[f])", float(conditions_geq[f]))
#             print("options = d_train[:, f] > conditions_geq[f]", options)

            if f in conditions_leq:
                options = options * (d_train[:, f] <= conditions_leq[f])
            
#             print('self.min[f]', self.min[f])
#             print('self.max[f]', self.max[f])
            """ if options.sum() -> options=[ False ... Fase] -> all of the regions on the sample are < conditions_geq[f] """
            if options.sum() == 0:
                min_ = conditions_geq.get(f, self.min[f])
                max_ = conditions_leq.get(f, self.max[f])
                to_rep = np.random.uniform(min_, max_, idx.sum())
            else:
                """ options  = for all the d_train checks in the f index is > the saved region (conditions_geq[f])
                    returning Ture or False.
                    
                    idx.sum() =  number of True in idx/
                    
                    idx = from the sample checks in the f index is <= the saved region (conditions_geq[f]),
                    returning Ture or False. Opposite to options.
                    
                    reaplace = True -> with replecemet -> Features can be selected multiple times -> bootstrapping. 
                    
                    len(train[options, f]) == options.sum() & idx.sum() == len(to_rep)
                    
                    """
                """ Randomelly selects with replacement from the training dataset where feature is > the saved reagio
                     and samples as many samples from the boostrap <=  to the saved reagion """
                
                """ So we are sampling fron the training dataset only if the condtion is grater, and sampling as many
                    times as in the sample it was <=. why? I would have thought if we want to be representative of the sample
                    we would have sample as manites they are >  not <= """
                to_rep = np.random.choice(train[options, f], idx.sum(), replace=True)
#                 print('train[options, f]', train[options, f])
#                 print('len(train[options, f])', len(train[options, f]))
#                 print('options.sum()', options.sum())
#                 print('idx.sum()', idx.sum())
#                 print('len(to_rep)', len(to_rep))
#                 print('to_rep', to_rep)
#                 print('len(train)', len(train))
        
#                 print("\n len(train)", len(train))
#                 print("options.sum() + idx.sum()", options.sum() + idx.sum())
                    
#                 options_2 = d_train[:, f] > conditions_geq[f]
#                 idx_2 = d_sample[:, f] <= conditions_geq[f]
                
#                 print("len(d_train[:, f])", len(d_train[:, f]))
#                 print("len(idx)", len(idx))
#                 print("len(options)", len(options))
#                  if options.all() == options_2.all() & idx.all() == idx_2.all():
#                     print("\nYessssss")
#                     print("options_2 = d_train[:, f] > conditions_geq[f], idx_2 = idx = d_sample[:, f] <= conditions_geq[f]")
#                     print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
#                     print("conditions_geq[f]", conditions_geq[f])
#                     print("d_train[:, f]", d_train[:, f])
#                     print("options", options)
#                     print("idx", idx)
#                     print("len(d_sample[:, f])", len(d_sample[:, f]))
#                     print("options.sum()", options.sum())
#                     print("idx.sum", idx.sum())
#                     print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
#                 else:
#                     print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
#                     print("\nNOOOOOO")
#                     print("conditions_geq[f]", conditions_geq[f])
#                     print("d_train[:, f]", d_train[:, f])
#                     print("options", options)
#                     print("idx", idx)
#                     print("len(d_sample[:, f])", len(d_sample[:, f]))
#                     print("options.sum()", options.sum())
#                     print("idx.sum", idx.sum())
#                     print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
                
            sample[idx, f] = to_rep
        
        """ Same thing as before but with conditions_leq """
        for f in conditions_leq:
            if f in conditions_geq:
                continue
            idx = d_sample[:, f] > conditions_leq[f]
            if idx.sum() == 0:
                continue
            options = d_train[:, f] <= conditions_leq[f]
            if options.sum() == 0:
                min_ = conditions_geq.get(f, self.min[f])
                max_ = conditions_leq.get(f, self.max[f])
                to_rep = np.random.uniform(min_, max_, idx.sum())
            else:
                to_rep = np.random.choice(train[options, f], idx.sum(),
                                          replace=True)
            sample[idx, f] = to_rep
        
#         print("sample", sample)
        return sample

    """ Used in get_sample_fn: def sample_fn """
    def as_html(self, exp, **kwargs):
        """bla"""
        exp_map = self.to_explanation_map(exp)

        def jsonize(x): return json.dumps(x)
        this_dir, _ = os.path.split(__file__)
        bundle = open(os.path.join(this_dir, 'bundle.js'), encoding='utf8').read()
        random_id = 'top_div' + id_generator()
        out = u'''<html>
        <meta http-equiv="content-type" content="text/html; charset=UTF8">
        <head><script>%s </script></head><body>''' % bundle
        out += u'''
        <div id="{random_id}" />
        <script>
            div = d3.select("#{random_id}");
            lime.RenderExplanationFrame(div,{label_names}, {predict_proba},
            {true_class}, {explanation}, {raw_data}, "tabular", {explanation_type});
        </script>'''.format(random_id=random_id,
                            label_names=jsonize(exp_map['labelNames']),
                            predict_proba=jsonize(exp_map['predictProba']),
                            true_class=jsonize(exp_map['trueClass']),
                            explanation=jsonize(exp_map['explanation']),
                            raw_data=jsonize(exp_map['rawData']),
                            explanation_type=jsonize(exp_map['explanationType']))
        out += u'</body></html>'
        return out
    

    """ Used in get_sample_fn: def sample_fn """
    def as_html(self, exp, **kwargs):
        """bla"""
        exp_map = self.to_explanation_map(exp)

#         def jsonize(x): return json.dumps(x)
#         this_dir, _ = os.path.split(__file__)
#         bundle = open(os.path.join(this_dir, 'bundle.js'), encoding='utf8').read()
#         random_id = 'top_div' + id_generator()
#         out = u'''<html>
#         <meta http-equiv="content-type" content="text/html; charset=UTF8">
#         <head><script>%s </script></head><body>''' % bundle
#         out += u'''
#         <div id="{random_id}" />
#         <script>
#             div = d3.select("#{random_id}");
#             lime.RenderExplanationFrame(div,{label_names}, {predict_proba},
#             {true_class}, {explanation}, {raw_data}, "tabular", {explanation_type});
#         </script>'''.format(random_id=random_id,
#                             label_names=jsonize(exp_map['labelNames']),
#                             predict_proba=jsonize(exp_map['predictProba']),
#                             true_class=jsonize(exp_map['trueClass']),
#                             explanation=jsonize(exp_map['explanation']),
#                             raw_data=jsonize(exp_map['rawData']),
#                             explanation_type=jsonize(exp_map['explanationType']))
#         out += u'</body></html>'
        return out
    
    """ Used in def explain_instance """
    
    """ This function returns: sample_fn, mapping
             mapping = makes a map dictionary saving if checking from 0 to 3 if the location of the quartile is <= or > 
             otherwise, for each of the features.
             sample_fn = function 
             """
    """ sample_fn returns: raw_data, data, labels,
            raw_data = samples 
            data = array of 1 and 0, 1 if the train_dataset is <=, > or == to the rigion...
            labels = array of 1 and 0, 1 if the predictions is the same as the true labels, 0 otherwise """
    def get_sample_fn(self, data_row, classifier_fn, desired_label=None):
        
        """ data_row = the instance we are interested """
        """predict_fn(x) its the prediction of the blackbox model"""
        def predict_fn(x):
            return classifier_fn(self.encoder_fn(x))
        true_label = desired_label
        if true_label is None:
            true_label = predict_fn(data_row.reshape(1, -1))[0]
#             print("data_row", data_row)
#             print("data_row.reshape(1, -1)", data_row.reshape(1, -1)[0])
#             print("true_label in loop", true_label)
        # must map present here to include categorical features (for conditions_eq), and numerical features for geq and leq
        """ makes a map dictionary saving if checking from 0 to 3 if the location of the quartile is <= or > otherwise,
            for each of the features"""
        mapping = {}
        """ Discretize the features into quartiles """
        data_row = self.disc.discretize(data_row.reshape(1, -1))[0]
        
#         print("data_row", data_row)
        
        """self.categorical_feature = self.ordinal_features: are list of the number of (n) features
            from 0 to n-1: [0, 1, 2, 3],  v goes from 0 to 3, v is the region of in the quartile domain"""
        """ I think I could chage the code to something more efficient"""
        for f in self.categorical_features:
            if f in self.ordinal_features:
#                 print("f", f)
#                 print("len(self.categorical_names[f])", len(self.categorical_names[f]))
                for v in range(len(self.categorical_names[f])):
                    idx = len(mapping)
# #                     print("mapping:", mapping)
# #                     print("idx = len(mapping):",  idx)
# #                     print("\n f, v:", f, v)
# #                     print("data_row:",  data_row)
# #                     print("data_row[f]:",  data_row[f])
# #                     print("data_row[f] <= v", data_row[f] <= v)
# #                     print("v != len(self.categorical_names[f]) - 1:",  v != len(self.categorical_names[f]) - 1)
                    if data_row[f] <= v and v != len(self.categorical_names[f]) - 1:
                        mapping[idx] = (f, 'leq', v)
                        # names[idx] = '%s <= %s' % (self.feature_names[f], v)
                    elif data_row[f] > v:
                        mapping[idx] = (f, 'geq', v)
                        # names[idx] = '%s > %s' % (self.feature_names[f], v)
            else:
#                 print("RICHARD THINKS I WILL NEVER BE PRINTED SINCE F WILL ALWAYS BE IN self.ordinal_features")
                idx = len(mapping)
                mapping[idx] = (f, 'eq', data_row[f])
            # names[idx] = '%s = %s' % (
            #     self.feature_names[f],
            #     self.categorical_names[f][int(data_row[f])])
        
        """ This function returns raw_data, data, labels,
            raw_data = samples 
            data = array of 1 and 0, 1 if the train_dataset is <=, > or == to the rigion...
            labels = array of 1 and 0, 1 if the predictions is the same as the true labels, 0 otherwise """
        def sample_fn(present, num_samples, compute_labels=True):
            conditions_eq = {}
            conditions_leq = {}
            conditions_geq = {}
            
#             print("present:", present)
            
            """ I think is the idex of the instance"""
            for x in present:
                f, op, v = mapping[x]
                if op == 'eq':
                    conditions_eq[f] = float(v)
                if op == 'leq':
                    if f not in conditions_leq:
                        conditions_leq[f] = v
                    conditions_leq[f] = min(conditions_leq[f], float(v))
                if op == 'geq':
                    if f not in conditions_geq:
                        conditions_geq[f] = v
                    conditions_geq[f] = max(conditions_geq[f], float(v))
            # conditions_eq = dict([(x, data_row[x]) for x in present])
#             print("conditions_eq", conditions_eq)
#             print("conditions_leq", conditions_leq)
#             print("conditions_geq", conditions_geq)

            """sample_from_train: 
                    Randomelly selects with replacement from the training dataset where feature is > the saved reagio
                    and samples as many samples from the boostrap <=  to the saved reagion, and vice versa"""
            raw_data = self.sample_from_train(
                conditions_eq, {}, conditions_geq, conditions_leq, num_samples)
            d_raw_data = self.disc.discretize(raw_data)
#             print("raw_data", raw_data)
#             print("d_raw_data", d_raw_data)
            data = np.zeros((num_samples, len(mapping)), int)
    

        
            """ Checks if the are in the same region, returns 1 or 0 if True or False
                why is this for? do we get an array of onlly 1 at the end? """
            """ len(len(mapping)) == 12 since 3*4 from the fucntions above wiouth 0?"""
#             print("len(mapping)", len(mapping))
            for i in mapping:
                f, op, v = mapping[i]
#                 print("len(d_raw_data[:, f])", len(d_raw_data[:, f]))
#                 print("data_row[f]", data_row[f])
                if op == 'eq':
                    data[:, i] = (d_raw_data[:, f] == data_row[f]).astype(int)
                if op == 'leq':
                    data[:, i] = (d_raw_data[:, f] <= v).astype(int)
                if op == 'geq':
                    data[:, i] = (d_raw_data[:, f] > v).astype(int)
            # data = (raw_data == data_row).astype(int)
            labels = []
            
            """ Compare if the predictions are correct of the instance"""
            """ However are we not comparing the prediction against the predictino since no true label was given 
                since true_label  was computed using the blackbox model """
            if compute_labels:
                labels = (predict_fn(raw_data) == true_label).astype(int)
            
#             print("labels", labels)
            return raw_data, data, labels
        
#         print("mapping", mapping)
        return sample_fn, mapping
         
    """ Used outside to explain an instance, this is the core functin, where all of the explanabily can happends ****"""
    def explain_instance(self, data_row, classifier_fn, threshold=0.95,
                          delta=0.1, tau=0.15, batch_size=100,
                          max_anchor_size=None,
                          desired_label=None,
                          beam_size=4, **kwargs):
        # It's possible to pass in max_anchor_size
        
        """ This function returns: sample_fn, mapping
        
                 mapping = makes a map dictionary saving if checking from 0 to 3 if the location of the quartile is <= or > 
                 otherwise, for each of the features.

                 sample_fn = a function 
         """
        sample_fn, mapping = self.get_sample_fn(
            data_row, classifier_fn, desired_label=desired_label)
        # return sample_fn, mapping
        
        
        """ Changing exp = anchor_base.AnchorBaseBeam.anchor_beam(...) to exp = AnchorBaseBeam.anchor_beam(...) """
        """ sample_fn returns raw_data, data, labels,
                raw_data = samples 
                data = array of 1 and 0, 1 if the train_dataset is <=, > or == to the rigion...
                labels = array of 1 and 0, 1 if the predictions is the same as the true labels, 0 otherwise """
        exp = AnchorBaseBeam.anchor_beam(
            sample_fn, delta=delta, epsilon=tau, batch_size=batch_size,
            desired_confidence=threshold, max_anchor_size=max_anchor_size,
            **kwargs)
        self.add_names_to_exp(data_row, exp, mapping)
        exp['instance'] = data_row
        exp['prediction'] = classifier_fn(self.encoder_fn(data_row.reshape(1, -1)))[0]
        
        """ cahnge: explanation = anchor_explanation.AnchorExplanation('tabular', exp, self.as_html)
                to: explanation = AnchorExplanation('tabular', exp, self.as_html) 
                Not imported but class defined """
        explanation = AnchorExplanation('tabular', exp, self.as_html)
        return explanation
    
    """ Used in explain_instance """
    def add_names_to_exp(self, data_row, hoeffding_exp, mapping):
        # TODO: precision recall is all wrong, coverage functions wont work
        # anymore due to ranges
        idxs = hoeffding_exp['feature']
        hoeffding_exp['names'] = []
        hoeffding_exp['feature'] = [mapping[idx][0] for idx in idxs]
        ordinal_ranges = {}
        for idx in idxs:
            f, op, v = mapping[idx]
            if op == 'geq' or op == 'leq':
                if f not in ordinal_ranges:
                    ordinal_ranges[f] = [float('-inf'), float('inf')]
            if op == 'geq':
                ordinal_ranges[f][0] = max(ordinal_ranges[f][0], v)
            if op == 'leq':
                ordinal_ranges[f][1] = min(ordinal_ranges[f][1], v)
        handled = set()
        for idx in idxs:
            f, op, v = mapping[idx]
            # v = data_row[f]
            if op == 'eq':
                fname = '%s = ' % self.feature_names[f]
                if f in self.categorical_names:
                    v = int(v)
                    if ('<' in self.categorical_names[f][v]
                            or '>' in self.categorical_names[f][v]):
                        fname = ''
                    fname = '%s%s' % (fname, self.categorical_names[f][v])
                else:
                    fname = '%s%.2f' % (fname, v)
            else:
                if f in handled:
                    continue
                geq, leq = ordinal_ranges[f]
                fname = ''
                geq_val = ''
                leq_val = ''
                if geq > float('-inf'):
                    if geq == len(self.categorical_names[f]) - 1:
                        geq = geq - 1
                    name = self.categorical_names[f][geq + 1]
                    if '<' in name:
                        geq_val = name.split()[0]
                    elif '>' in name:
                        geq_val = name.split()[-1]
                if leq < float('inf'):
                    name = self.categorical_names[f][leq]
                    if leq == 0:
                        leq_val = name.split()[-1]
                    elif '<' in name:
                        leq_val = name.split()[-1]
                if leq_val and geq_val:
                    fname = '%s < %s <= %s' % (geq_val, self.feature_names[f],
                                               leq_val)
                elif leq_val:
                    fname = '%s <= %s' % (self.feature_names[f], leq_val)
                elif geq_val:
                    fname = '%s > %s' % (self.feature_names[f], geq_val)
                handled.add(f)
            hoeffding_exp['names'].append(fname)
   
#     """~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  NOT NEEDED  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"""
#     """ This functions in not used at all in the code """
#     def transform_to_examples(self, examples, features_in_anchor=[],
#                               predicted_label=None):
#         ret_obj = []
#         if len(examples) == 0:
#             return ret_obj
#         weights = [int(predicted_label) if x in features_in_anchor else -1
#                    for x in range(examples.shape[1])]
#         examples = self.disc.discretize(examples)
#         for ex in examples:
#             values = [self.categorical_names[i][int(ex[i])]
#                       if i in self.categorical_features
#                       else ex[i] for i in range(ex.shape[0])]
#             ret_obj.append(list(zip(self.feature_names, values, weights)))
#         return ret_obj
    

#     """~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ END - NOT NEEDED  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"""

# # """ We do not use  def transform_to_examples and  def to_explanation_map(self, exp)"""

In [78]:
class QuartileDiscretizer(BaseDiscretizer):
    def __init__(self, data, categorical_features, feature_names, labels=None, random_state=None):

        BaseDiscretizer.__init__(self, data, categorical_features,
                                 feature_names, labels=labels,
                                 random_state=random_state)

    def bins(self, data, labels):
        bins = []
        for feature in self.to_discretize:
            qts = np.array(np.percentile(data[:, feature], [25, 50, 75]))
            bins.append(qts)
        return bins

In [82]:
train = np.array(X)[:, 0:8]
labels_train = np.array(y)

test = np.array(X_test)[:, 0:8]
labels_test = np.array(y_test)

In [83]:
feature_names

['0', '1', '2', '3', '4', '5', '6', '7']

In [84]:
explainer_1 = AnchorTabularExplainer(
    target_names,
    feature_names,
    train,
    train_label = labels_train,
    discretizer = 'quartile')

explainer_2 = AnchorTabularExplainer(
    target_names,
    feature_names,
    train,
    train_label = labels_train,
    discretizer = 'decile')

explainer_3 = AnchorTabularExplainer(
    target_names,
    feature_names,
    train,
    train_label = labels_train,
    discretizer = 'entropy')

explainer_4 = AnchorTabularExplainer(
    target_names,
    feature_names,
    train,
    train_label = labels_train,
    discretizer = 'GaussianMixtureModels')

C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Po

In [85]:
results = {"Number" : [], "Name": [], 
               "quartile mean precision": [], "quartile mean coverage": [], 
               "decile mean precision": [], "decile mean coverage": [],
               "entropy mean precision": [], "entropy mean coverage": [],
               "GaussianMixtureModels mean precision": [], "GaussianMixtureModels mean coverage": []}

In [86]:
r = 1

In [87]:
exp = explainer_4.explain_instance(test[1], blackbox_model.predict, threshold=0.95)
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())

print('Coverage: %.2f' % exp.coverage())

Anchor: 1 <= 0.00 AND 4 > 0.00 AND 3 <= 3.00 AND 6 <= 4.93 AND 0 <= 40.76
Precision: 0.95
Coverage: 0.00


In [89]:
exp = explainer_3.explain_instance(test[1], blackbox_model.predict, threshold=0.95)
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())

print('Coverage: %.2f' % exp.coverage())

Anchor: 0.50 < 3 <= 2.50 AND 0.50 < 0 <= 27.50 AND 1 <= 2.50 AND 0.50 < 4 <= 2.50 AND 2 <= 833.50 AND 7 <= 4.50 AND 6 <= 4.50
Precision: 0.91
Coverage: 0.00


In [90]:
exp = explainer_2.explain_instance(test[1], blackbox_model.predict, threshold=0.95)
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())

print('Coverage: %.2f' % exp.coverage())

Anchor: 1 <= 1.00 AND 0.00 < 3 <= 2.00 AND 6 <= 4.00 AND 4 <= 8.00 AND 0 <= 32.00
Precision: 0.96
Coverage: 0.01


In [91]:
exp = explainer_1.explain_instance(test[1], blackbox_model.predict, threshold=0.95)
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())

print('Coverage: %.2f' % exp.coverage())

Anchor: 3 <= 2.00 AND 1 <= 2.00
Precision: 0.82
Coverage: 0.43


In [102]:
import statistics
def getMeanPrecisionCoverage(explainer, results, discretizer_name):
    test_precision = []
    test_coverage = []
    for idx in range(len(test)):
        exp = explainer.explain_instance(test[idx], blackbox_model.predict, threshold=0.95)
        
        test_precision.append(exp.precision())
        test_coverage.append(exp.coverage())
#         print('exp.precision()', exp.precision())
#         print('exp.coverage()', exp.coverage())
        

    results[discretizer_name + " mean precision"].append(statistics.mean(test_precision))
    results[discretizer_name + " mean coverage"].append(statistics.mean(test_coverage))
#     print('Anchor: %s' % (' AND '.join(exp.names())))
#     print('Precision: %.2f' % exp.precision())
#     print('Coverage: %.2f' % exp.coverage())
    print(results)

In [103]:
results = {"Number" : [], "Name": [], 
               "quartile mean precision": [], "quartile mean coverage": [], 
               "decile mean precision": [], "decile mean coverage": [],
               "entropy mean precision": [], "entropy mean coverage": [],
               "GaussianMixtureModels mean precision": [], "GaussianMixtureModels mean coverage": []}

In [104]:
def testDataset():
    
    
    
    
#     datasets = data_dictionary["Data"]
    count = 0
#     for data in datasets:
        
#         print("dataset number", count)
        
#         results["Number"].append(count)
#         results["Name"].append(data_dictionary["Name"][count])


#         train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(X_test, y_test, train_size=0.80)
    blackbox_model = sklearn.ensemble.RandomForestClassifier(n_estimators=20)
    blackbox_model.fit(train, labels_train)

    explainer_1 = AnchorTabularExplainer(
        target_names,
        feature_names,
        train,
        train_label = labels_train,
        discretizer = 'quartile')

    explainer_2 = AnchorTabularExplainer(
        target_names,
        feature_names,
        train,
        train_label = labels_train,
        discretizer = 'decile')

    explainer_3 = AnchorTabularExplainer(
        target_names,
        feature_names,
        train,
        train_label = labels_train,
        discretizer = 'entropy')

    explainer_4 = AnchorTabularExplainer(
        target_names,
        feature_names,
        train,
        train_label = labels_train,
        discretizer = 'GaussianMixtureModels')


    getMeanPrecisionCoverage(explainer_4, results, "GaussianMixtureModels")
    getMeanPrecisionCoverage(explainer_1, results, "quartile")
    getMeanPrecisionCoverage(explainer_2, results, "decile")
    getMeanPrecisionCoverage(explainer_3, results, "entropy")


    print("results", results)
    count += 1
    
    return results

In [105]:
train = np.array(X)
labels_train = np.array(y)

test1 = np.array(X_test)
labels_test1 = np.array(y_test)

In [106]:
test = test1[:100]
labels_test = labels_test1[:100]
results_adult = testDataset()

C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Po

{'Number': [], 'Name': [], 'quartile mean precision': [], 'quartile mean coverage': [], 'decile mean precision': [], 'decile mean coverage': [], 'entropy mean precision': [], 'entropy mean coverage': [], 'GaussianMixtureModels mean precision': [0.964062686274484], 'GaussianMixtureModels mean coverage': [0.126633]}
{'Number': [], 'Name': [], 'quartile mean precision': [0.8734933547041843], 'quartile mean coverage': [0.159416], 'decile mean precision': [], 'decile mean coverage': [], 'entropy mean precision': [], 'entropy mean coverage': [], 'GaussianMixtureModels mean precision': [0.964062686274484], 'GaussianMixtureModels mean coverage': [0.126633]}
{'Number': [], 'Name': [], 'quartile mean precision': [0.8734933547041843], 'quartile mean coverage': [0.159416], 'decile mean precision': [0.9198142309863904], 'decile mean coverage': [0.138112], 'entropy mean precision': [], 'entropy mean coverage': [], 'GaussianMixtureModels mean precision': [0.964062686274484], 'GaussianMixtureModels me

In [107]:
test = test1[100:200]
labels_test = labels_test1[100:200]
results_adult = testDataset()

C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Po

{'Number': [], 'Name': [], 'quartile mean precision': [0.8734933547041843], 'quartile mean coverage': [0.159416], 'decile mean precision': [0.9198142309863904], 'decile mean coverage': [0.138112], 'entropy mean precision': [0.9295078641705959], 'entropy mean coverage': [0.134798], 'GaussianMixtureModels mean precision': [0.964062686274484, 0.966787906868761], 'GaussianMixtureModels mean coverage': [0.126633, 0.143927]}
{'Number': [], 'Name': [], 'quartile mean precision': [0.8734933547041843, 0.9021624157790343], 'quartile mean coverage': [0.159416, 0.132466], 'decile mean precision': [0.9198142309863904], 'decile mean coverage': [0.138112], 'entropy mean precision': [0.9295078641705959], 'entropy mean coverage': [0.134798], 'GaussianMixtureModels mean precision': [0.964062686274484, 0.966787906868761], 'GaussianMixtureModels mean coverage': [0.126633, 0.143927]}
{'Number': [], 'Name': [], 'quartile mean precision': [0.8734933547041843, 0.9021624157790343], 'quartile mean coverage': [0

In [108]:
test = test1[200:300]
labels_test = labels_test1[200:300]
results_adult = testDataset()

C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Po

{'Number': [], 'Name': [], 'quartile mean precision': [0.8734933547041843, 0.9021624157790343], 'quartile mean coverage': [0.159416, 0.132466], 'decile mean precision': [0.9198142309863904, 0.935485698903857], 'decile mean coverage': [0.138112, 0.14564], 'entropy mean precision': [0.9295078641705959, 0.9407129830299467], 'entropy mean coverage': [0.134798, 0.158045], 'GaussianMixtureModels mean precision': [0.964062686274484, 0.966787906868761, 0.9649019625507983], 'GaussianMixtureModels mean coverage': [0.126633, 0.143927, 0.137842]}
{'Number': [], 'Name': [], 'quartile mean precision': [0.8734933547041843, 0.9021624157790343, 0.8845284480733878], 'quartile mean coverage': [0.159416, 0.132466, 0.1528], 'decile mean precision': [0.9198142309863904, 0.935485698903857], 'decile mean coverage': [0.138112, 0.14564], 'entropy mean precision': [0.9295078641705959, 0.9407129830299467], 'entropy mean coverage': [0.134798, 0.158045], 'GaussianMixtureModels mean precision': [0.964062686274484, 0

In [109]:
test = test1[300:400]
labels_test = labels_test1[300:400]
results_adult = testDataset()

C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Po

{'Number': [], 'Name': [], 'quartile mean precision': [0.8734933547041843, 0.9021624157790343, 0.8845284480733878], 'quartile mean coverage': [0.159416, 0.132466, 0.1528], 'decile mean precision': [0.9198142309863904, 0.935485698903857, 0.9202325780066681], 'decile mean coverage': [0.138112, 0.14564, 0.137027], 'entropy mean precision': [0.9295078641705959, 0.9407129830299467, 0.9411758916810502], 'entropy mean coverage': [0.134798, 0.158045, 0.142472], 'GaussianMixtureModels mean precision': [0.964062686274484, 0.966787906868761, 0.9649019625507983, 0.9687371508670325], 'GaussianMixtureModels mean coverage': [0.126633, 0.143927, 0.137842, 0.120606]}
{'Number': [], 'Name': [], 'quartile mean precision': [0.8734933547041843, 0.9021624157790343, 0.8845284480733878, 0.8715482940341757], 'quartile mean coverage': [0.159416, 0.132466, 0.1528, 0.124005], 'decile mean precision': [0.9198142309863904, 0.935485698903857, 0.9202325780066681], 'decile mean coverage': [0.138112, 0.14564, 0.137027]

In [110]:
test = test1[400:500]
labels_test = labels_test1[400:500]
results_adult = testDataset()

C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Po

{'Number': [], 'Name': [], 'quartile mean precision': [0.8734933547041843, 0.9021624157790343, 0.8845284480733878, 0.8715482940341757], 'quartile mean coverage': [0.159416, 0.132466, 0.1528, 0.124005], 'decile mean precision': [0.9198142309863904, 0.935485698903857, 0.9202325780066681, 0.91643348480619], 'decile mean coverage': [0.138112, 0.14564, 0.137027, 0.128096], 'entropy mean precision': [0.9295078641705959, 0.9407129830299467, 0.9411758916810502, 0.9363317535821238], 'entropy mean coverage': [0.134798, 0.158045, 0.142472, 0.13396], 'GaussianMixtureModels mean precision': [0.964062686274484, 0.966787906868761, 0.9649019625507983, 0.9687371508670325, 0.9643316280046956], 'GaussianMixtureModels mean coverage': [0.126633, 0.143927, 0.137842, 0.120606, 0.151235]}
{'Number': [], 'Name': [], 'quartile mean precision': [0.8734933547041843, 0.9021624157790343, 0.8845284480733878, 0.8715482940341757, 0.8903874573949927], 'quartile mean coverage': [0.159416, 0.132466, 0.1528, 0.124005, 0.1

In [111]:
test = test1[500:600]
labels_test = labels_test1[500:600]
results_adult = testDataset()

C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Po

{'Number': [], 'Name': [], 'quartile mean precision': [0.8734933547041843, 0.9021624157790343, 0.8845284480733878, 0.8715482940341757, 0.8903874573949927], 'quartile mean coverage': [0.159416, 0.132466, 0.1528, 0.124005, 0.156117], 'decile mean precision': [0.9198142309863904, 0.935485698903857, 0.9202325780066681, 0.91643348480619, 0.9239993533374442], 'decile mean coverage': [0.138112, 0.14564, 0.137027, 0.128096, 0.159434], 'entropy mean precision': [0.9295078641705959, 0.9407129830299467, 0.9411758916810502, 0.9363317535821238, 0.9388186128756045], 'entropy mean coverage': [0.134798, 0.158045, 0.142472, 0.13396, 0.152444], 'GaussianMixtureModels mean precision': [0.964062686274484, 0.966787906868761, 0.9649019625507983, 0.9687371508670325, 0.9643316280046956, 0.9661992548085844], 'GaussianMixtureModels mean coverage': [0.126633, 0.143927, 0.137842, 0.120606, 0.151235, 0.122005]}
{'Number': [], 'Name': [], 'quartile mean precision': [0.8734933547041843, 0.9021624157790343, 0.8845284

In [116]:
len(labels_test1)

16281

In [117]:
test = test1[600:800]
labels_test = labels_test1[600:800]
results_adult = testDataset()

C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Po

{'Number': [], 'Name': [], 'quartile mean precision': [0.8734933547041843, 0.9021624157790343, 0.8845284480733878, 0.8715482940341757, 0.8903874573949927, 0.9034954121072202], 'quartile mean coverage': [0.159416, 0.132466, 0.1528, 0.124005, 0.156117, 0.120397], 'decile mean precision': [0.9198142309863904, 0.935485698903857, 0.9202325780066681, 0.91643348480619, 0.9239993533374442, 0.9421492560461339], 'decile mean coverage': [0.138112, 0.14564, 0.137027, 0.128096, 0.159434, 0.123785], 'entropy mean precision': [0.9295078641705959, 0.9407129830299467, 0.9411758916810502, 0.9363317535821238, 0.9388186128756045, 0.9427826159036578], 'entropy mean coverage': [0.134798, 0.158045, 0.142472, 0.13396, 0.152444, 0.124292], 'GaussianMixtureModels mean precision': [0.964062686274484, 0.966787906868761, 0.9649019625507983, 0.9687371508670325, 0.9643316280046956, 0.9661992548085844, 0.9671578261725734], 'GaussianMixtureModels mean coverage': [0.126633, 0.143927, 0.137842, 0.120606, 0.151235, 0.122

In [118]:
test = test1[800:900]
labels_test = labels_test1[800:900]
results_adult = testDataset()

C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\richa\Anaconda3\envs\torch\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Po

KeyboardInterrupt: 

In [ ]:
test = test1[900:1000]
labels_test = labels_test1[900:1000]
results_adult = testDataset()

In [ ]:
test = test1[1000:1500]
labels_test = labels_test1[1000:1500]
results_adult = testDataset()

In [ ]:
test = test1[1500:2000]
labels_test = labels_test1[1500:2000]
results_adult = testDataset()

In [12]:
results['GaussianMixtureModels mean coverage']

[0.126633, 0.143927, 0.137842, 0.120606, 0.151235, 0.122005, 0.131659]

In [1]:
results = {'Number': [], 'Name': [], 'quartile mean precision': [0.8734933547041843, 0.9021624157790343, 0.8845284480733878, 0.8715482940341757, 0.8903874573949927, 0.9034954121072202, 0.8848852313910525], 'quartile mean coverage': [0.159416, 0.132466, 0.1528, 0.124005, 0.156117, 0.120397, 0.1235025], 'decile mean precision': [0.9198142309863904, 0.935485698903857, 0.9202325780066681, 0.91643348480619, 0.9239993533374442, 0.9421492560461339, 0.9257869493229853], 'decile mean coverage': [0.138112, 0.14564, 0.137027, 0.128096, 0.159434, 0.123785, 0.1242235], 'entropy mean precision': [0.9295078641705959, 0.9407129830299467, 0.9411758916810502, 0.9363317535821238, 0.9388186128756045, 0.9427826159036578, 0.93973720927555], 'entropy mean coverage': [0.134798, 0.158045, 0.142472, 0.13396, 0.152444, 0.124292, 0.1356115], 'GaussianMixtureModels mean precision': [0.964062686274484, 0.966787906868761, 0.9649019625507983, 0.9687371508670325, 0.9643316280046956, 0.9661992548085844, 0.9671578261725734], 'GaussianMixtureModels mean coverage': [0.126633, 0.143927, 0.137842, 0.120606, 0.151235, 0.122005, 0.131659]}

In [6]:
import statistics
statistics.mean(results['quartile mean precision']), statistics.mean(results['quartile mean coverage'])

(0.887214373354864, 0.1383862142857143)

In [8]:
statistics.mean(results['decile mean precision']),  statistics.mean(results['decile mean coverage'])

(0.9262716502013812, 0.1366167857142857)

In [10]:
statistics.mean(results['entropy mean precision']),  statistics.mean(results['entropy mean coverage'])

(0.9384381329312185, 0.1402317857142857)

In [11]:
statistics.mean(results['GaussianMixtureModels mean precision']),  statistics.mean(results['GaussianMixtureModels mean coverage'])

(0.9660254879352757, 0.1334152857142857)

In [ ]:
decile entropy GaussianMixtureModels

In [ ]:
statistics.mean(results_tests['quartile mean precision']), statistics.mean(results_tests['quartile mean coverage'])

In [ ]:
statistics.mean(results_tests['decile mean precision']), statistics.mean(results_tests['decile mean coverage'])

In [ ]:
statistics.mean(results_tests['entropy mean precision']), statistics.mean(results_tests['entropy mean coverage'])

In [ ]:
statistics.mean(results_tests['GaussianMixtureModels mean precision']), statistics.mean(results_tests['GaussianMixtureModels mean coverage'])

In [ ]:
test = test[1000:1500]
labels_test = labels_test[1000:1500]

In [ ]:
{'Number': [], 'Name': [], 'quartile mean precision': [0.8734933547041843, 0.9021624157790343, 0.8845284480733878, 0.8715482940341757, 0.8903874573949927], 'quartile mean coverage': [0.159416, 0.132466, 0.1528, 0.124005, 0.156117], 'decile mean precision': [0.9198142309863904, 0.935485698903857, 0.9202325780066681, 0.91643348480619, 0.9239993533374442], 'decile mean coverage': [0.138112, 0.14564, 0.137027, 0.128096, 0.159434], 'entropy mean precision': [0.9295078641705959, 0.9407129830299467, 0.9411758916810502, 0.9363317535821238, 0.9388186128756045], 'entropy mean coverage': [0.134798, 0.158045, 0.142472, 0.13396, 0.152444], 'GaussianMixtureModels mean precision': [0.964062686274484, 0.966787906868761, 0.9649019625507983, 0.9687371508670325, 0.9643316280046956], 'GaussianMixtureModels mean coverage': [0.126633, 0.143927, 0.137842, 0.120606, 0.151235]}
results {'Number': [], 'Name': [], 'quartile mean precision': [0.8734933547041843, 0.9021624157790343, 0.8845284480733878, 0.8715482940341757, 0.8903874573949927], 'quartile mean coverage': [0.159416, 0.132466, 0.1528, 0.124005, 0.156117], 'decile mean precision': [0.9198142309863904, 0.935485698903857, 0.9202325780066681, 0.91643348480619, 0.9239993533374442], 'decile mean coverage': [0.138112, 0.14564, 0.137027, 0.128096, 0.159434], 'entropy mean precision': [0.9295078641705959, 0.9407129830299467, 0.9411758916810502, 0.9363317535821238, 0.9388186128756045], 'entropy mean coverage': [0.134798, 0.158045, 0.142472, 0.13396, 0.152444], 'GaussianMixtureModels mean precision': [0.964062686274484, 0.966787906868761, 0.9649019625507983, 0.9687371508670325, 0.9643316280046956], 'GaussianMixtureModels mean coverage': [0.126633, 0.143927, 0.137842, 0.120606, 0.151235]}

In [ ]:
# quartile
# Anchor: 5 <= 0.00
# Precision: 0.96
# Coverage: 0.32
# quartile
# Anchor: 8 > 0.00 AND 10 <= 1.00 AND 11 <= 0.00 AND 4 <= 1.00 AND 12 <= 3.00 AND 1 <= 2.00 AND 9 <= 1.00 AND 3 <= 1.00 AND 13 <= 0.00 AND 0 <= 11.00
# Precision: 0.91
# Coverage: 0.01
# quartile
# Anchor: 1.00 < 4 <= 2.00 AND 10 <= 1.00 AND 8 > 0.00 AND 11 <= 0.00
# Precision: 0.95
# Coverage: 0.00
# quartile
# Anchor: 13 <= 0.00 AND 0.00 < 5 <= 1.00 AND 0.00 < 7 <= 1.00 AND 9 <= 0.00 AND 8 <= 0.00 AND 6 <= 2.00 AND 0 <= 11.00 AND 12 <= 9.00 AND 2 <= 3940.00 AND 3 <= 5.00 AND 4 <= 5.00
# Precision: 0.46
# Coverage: 0.01
# quartile
# Anchor: 5 <= 0.00 AND 9 > 0.00
# Precision: 0.97
# Coverage: 0.15
# quartile
# Anchor: 5 <= 0.00 AND 10 <= 1.00
# Precision: 0.98
# Coverage: 0.32
# quartile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.98
# Coverage: 0.19
# quartile
# Anchor: 6 > 4.00 AND 8 > 0.00 AND 3 > 2.00 AND 11 <= 0.00 AND 12 <= 3.00 AND 1 > 2.00 AND 4 <= 5.00 AND 0 <= 22.00 AND 13 <= 0.00
# Precision: 0.91
# Coverage: 0.00
# quartile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.98
# Coverage: 0.18
# quartile
# Anchor: 7 <= 3.00 AND 0.00 < 5 <= 1.00 AND 6 <= 7.00
# Precision: 0.48
# Coverage: 0.37
# quartile
# Anchor: 10 > 1.00 AND 0.00 < 5 <= 1.00 AND 0.00 < 7 <= 1.00 AND 6 <= 2.00 AND 3 <= 1.00 AND 9 <= 0.00 AND 12 <= 9.00 AND 13 <= 0.00 AND 0 <= 11.00 AND 4 <= 1.00 AND 1 <= 2.00
# Precision: 0.78
# Coverage: 0.00
# quartile
# Anchor: 6 > 4.00 AND 4 > 5.00 AND 10 <= 1.00 AND 8 > 0.00 AND 11 <= 0.00
# Precision: 0.96
# Coverage: 0.01
# quartile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.98
# Coverage: 0.19
# quartile
# Anchor: 8 > 0.00 AND 10 <= 1.00 AND 11 <= 0.00 AND 1 <= 2.00 AND 11.00 < 0 <= 22.00 AND 3 <= 2.00 AND 6 > 2.00 AND 9 <= 1.00 AND 2 <= 14408.00 AND 4 <= 2.00 AND 12 <= 9.00 AND 13 <= 0.00
# Precision: 0.93
# Coverage: 0.00
# quartile
# Anchor: 10 > 1.00 AND 0.00 < 5 <= 1.00 AND 9 <= 0.00 AND 0.00 < 7 <= 1.00 AND 6 <= 2.00 AND 4 <= 1.00 AND 2 <= 3940.00 AND 0 <= 22.00 AND 13 <= 0.00 AND 11 <= 0.00 AND 3 <= 1.00 AND 12 <= 9.00
# Precision: 0.78
# Coverage: 0.00
# quartile
# Anchor: 6 > 7.00 AND 8 > 0.00 AND 10 <= 1.00 AND 13 <= 0.00
# Precision: 0.96
# Coverage: 0.03
# quartile
# Anchor: 5 <= 0.00 AND 10 <= 1.00
# Precision: 0.98
# Coverage: 0.31
# quartile
# Anchor: 7 > 3.00 AND 9 > 0.00
# Precision: 0.97
# Coverage: 0.09
# quartile
# Anchor: 7 > 1.00 AND 5 > 1.00 AND 9 > 0.00
# Precision: 0.97
# Coverage: 0.08
# quartile
# Anchor: 6 > 4.00 AND 8 > 0.00 AND 10 <= 1.00 AND 4 > 5.00 AND 0 > 11.00
# Precision: 0.95
# Coverage: 0.01
# quartile
# Anchor: 9 <= 0.00 AND 0.00 < 5 <= 1.00 AND 0.00 < 7 <= 1.00 AND 13 <= 0.00 AND 0 <= 34.00
# Precision: 0.48
# Coverage: 0.27
# quartile
# Anchor: 5 <= 0.00
# Precision: 0.96
# Coverage: 0.33
# quartile
# Anchor: 7 > 1.00 AND 5 > 1.00
# Precision: 0.95
# Coverage: 0.10
# quartile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 1.00
# Coverage: 0.19
# quartile
# Anchor: 0.00 < 5 <= 1.00 AND 9 <= 0.00 AND 3 > 2.00 AND 7 <= 1.00 AND 4 > 2.00
# Precision: 0.46
# Coverage: 0.19
# quartile
# Anchor: 6 > 4.00 AND 8 > 0.00 AND 11 <= 0.00 AND 12 > 9.00 AND 0 > 11.00
# Precision: 0.90
# Coverage: 0.01
# quartile
# Anchor: 5 <= 0.00 AND 10 <= 1.00
# Precision: 0.97
# Coverage: 0.32
# quartile
# Anchor: 7 > 1.00 AND 5 > 1.00 AND 10 <= 1.00
# Precision: 0.96
# Coverage: 0.10
# quartile
# Anchor: 6 > 4.00 AND 8 > 0.00 AND 10 <= 1.00 AND 11 <= 0.00 AND 12 > 3.00 AND 1 <= 2.00 AND 0 > 11.00
# Precision: 0.93
# Coverage: 0.01
# quartile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.98
# Coverage: 0.19
# quartile
# Anchor: 9 <= 0.00 AND 0.00 < 5 <= 1.00 AND 13 <= 0.00 AND 0.00 < 7 <= 1.00
# Precision: 0.44
# Coverage: 0.37
# quartile
# Anchor: 7 > 1.00 AND 5 > 1.00 AND 9 > 0.00
# Precision: 0.96
# Coverage: 0.08
# quartile
# Anchor: 5 <= 0.00
# Precision: 0.96
# Coverage: 0.33
# quartile
# Anchor: 3 > 2.00 AND 8 > 0.00 AND 10 <= 1.00 AND 4 <= 5.00 AND 1 <= 2.00 AND 11 <= 0.00 AND 12 <= 0.00 AND 9 <= 1.00 AND 6 <= 2.00 AND 0 <= 22.00 AND 2 <= 14408.00
# Precision: 0.96
# Coverage: 0.00
# quartile
# Anchor: 7 > 1.00 AND 6 > 7.00
# Precision: 0.96
# Coverage: 0.07
# quartile
# Anchor: 8 > 0.00 AND 12 <= 0.00 AND 10 <= 1.00 AND 3 <= 2.00 AND 6 > 2.00 AND 1 <= 2.00 AND 11 <= 0.00 AND 13 <= 0.00 AND 4 <= 2.00 AND 2 <= 14408.00 AND 9 <= 1.00
# Precision: 0.93
# Coverage: 0.01
# quartile
# Anchor: 0.00 < 5 <= 1.00 AND 0.00 < 7 <= 1.00 AND 6 <= 7.00 AND 2 <= 14408.00
# Precision: 0.50
# Coverage: 0.26
# quartile
# Anchor: 5 <= 0.00
# Precision: 0.96
# Coverage: 0.33
# quartile
# Anchor: 5 <= 0.00
# Precision: 0.95
# Coverage: 0.32
# quartile
# Anchor: 5 <= 0.00 AND 10 <= 1.00
# Precision: 0.97
# Coverage: 0.31
# quartile
# Anchor: 6 > 4.00 AND 8 > 0.00 AND 10 <= 1.00 AND 4 > 5.00 AND 11 <= 0.00 AND 1 <= 2.00
# Precision: 0.96
# Coverage: 0.01
# quartile
# Anchor: 6 > 7.00 AND 8 > 0.00 AND 10 <= 1.00 AND 4 > 5.00 AND 12 > 9.00
# Precision: 0.96
# Coverage: 0.00
# quartile
# Anchor: 4 <= 2.00 AND 8 > 0.00 AND 10 <= 1.00 AND 12 <= 0.00 AND 11 <= 0.00 AND 1 <= 2.00 AND 9 <= 1.00 AND 0 <= 11.00 AND 3 <= 5.00 AND 13 <= 0.00 AND 2 <= 8661.00 AND 6 <= 2.00
# Precision: 0.95
# Coverage: 0.00
# quartile
# Anchor: 8 > 0.00 AND 10 <= 1.00 AND 4 <= 2.00 AND 6 > 2.00 AND 12 <= 0.00 AND 11 <= 0.00 AND 2 <= 14408.00 AND 1 <= 2.00 AND 9 <= 1.00 AND 13 <= 0.00 AND 3 <= 5.00
# Precision: 0.94
# Coverage: 0.01
# quartile
# Anchor: 5 <= 0.00
# Precision: 0.95
# Coverage: 0.33
# quartile
# Anchor: 7 > 3.00 AND 9 > 0.00
# Precision: 0.97
# Coverage: 0.10
# quartile
# Anchor: 11 <= 0.00 AND 0.00 < 5 <= 1.00 AND 6 <= 7.00 AND 9 <= 1.00 AND 0 <= 34.00
# Precision: 0.46
# Coverage: 0.25
# quartile
# Anchor: 10 > 1.00 AND 0.00 < 5 <= 1.00 AND 0.00 < 7 <= 1.00 AND 2.00 < 3 <= 5.00 AND 9 <= 0.00 AND 2 <= 3940.00 AND 4 <= 5.00 AND 13 <= 0.00 AND 1 <= 2.00 AND 0.00 < 12 <= 3.00 AND 22.00 < 0 <= 34.00 AND 11 <= 0.00 AND 6 > 7.00 AND 8 > 0.00
# Precision: 0.74
# Coverage: 0.00
# quartile
# Anchor: 4 <= 5.00 AND 5 > 0.00 AND 2 <= 3940.00 AND 0.00 < 7 <= 3.00 AND 8 <= 0.00
# Precision: 0.39
# Coverage: 0.09
# quartile
# Anchor: 7 > 1.00 AND 5 > 1.00 AND 6 > 7.00
# Precision: 0.98
# Coverage: 0.02
# quartile
# Anchor: 5 <= 0.00 AND 10 <= 1.00
# Precision: 0.98
# Coverage: 0.32
# quartile
# Anchor: 5 > 0.00 AND 7 <= 1.00 AND 9 <= 0.00 AND 8 <= 0.00 AND 13 <= 0.00 AND 2 <= 8661.00
# Precision: 0.40
# Coverage: 0.21
# quartile
# Anchor: 5 <= 0.00 AND 10 <= 1.00
# Precision: 0.98
# Coverage: 0.30
# quartile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.99
# Coverage: 0.18
# quartile
# Anchor: 7 > 1.00 AND 6 > 7.00
# Precision: 0.95
# Coverage: 0.07
# quartile
# Anchor: 5 <= 0.00 AND 10 <= 1.00
# Precision: 0.98
# Coverage: 0.30
# quartile
# Anchor: 9 <= 1.00 AND 0.00 < 5 <= 1.00
# Precision: 0.45
# Coverage: 0.46
# quartile
# Anchor: 10 > 1.00 AND 0.00 < 5 <= 1.00 AND 0.00 < 7 <= 1.00 AND 13 <= 0.00 AND 9 <= 0.00 AND 0 <= 11.00 AND 0.00 < 12 <= 3.00 AND 1 <= 2.00 AND 11 <= 0.00 AND 8 > 0.00 AND 2 <= 8661.00 AND 3 > 2.00
# Precision: 0.70
# Coverage: 0.00
# quartile
# Anchor: 10 > 1.00 AND 0.00 < 5 <= 1.00 AND 3 <= 1.00 AND 0.00 < 7 <= 1.00 AND 9 <= 0.00 AND 4 <= 1.00 AND 2 <= 3940.00 AND 13 <= 0.00 AND 22.00 < 0 <= 34.00 AND 0.00 < 12 <= 3.00 AND 8 > 0.00 AND 1 > 2.00 AND 11 <= 0.00 AND 6 > 4.00
# Precision: 0.73
# Coverage: 0.00
# quartile
# Anchor: 6 > 7.00 AND 8 > 0.00 AND 10 <= 1.00 AND 11 <= 0.00 AND 1 <= 2.00 AND 0 > 22.00 AND 9 <= 1.00 AND 4 <= 2.00 AND 12 <= 3.00
# Precision: 0.95
# Coverage: 0.00
# quartile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.99
# Coverage: 0.18
# quartile
# Anchor: 0 <= 34.00 AND 5 > 0.00 AND 9 <= 0.00 AND 3 > 2.00 AND 7 > 0.00 AND 13 <= 0.00
# Precision: 0.48
# Coverage: 0.12
# quartile
# Anchor: 5 <= 0.00
# Precision: 0.95
# Coverage: 0.32
# quartile
# Anchor: 5 <= 0.00
# Precision: 0.95
# Coverage: 0.33
# quartile
# Anchor: 7 > 1.00 AND 5 > 1.00
# Precision: 0.95
# Coverage: 0.10
# quartile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.98
# Coverage: 0.18
# quartile
# Anchor: 10 > 1.00 AND 0.00 < 5 <= 1.00 AND 0.00 < 7 <= 1.00 AND 4.00 < 6 <= 7.00 AND 9 <= 0.00 AND 4 <= 2.00 AND 13 <= 0.00 AND 1 > 2.00 AND 3 <= 2.00 AND 2 <= 3940.00
# Precision: 0.76
# Coverage: 0.00
# quartile
# Anchor: 5 <= 0.00 AND 10 <= 1.00
# Precision: 0.98
# Coverage: 0.32
# quartile
# Anchor: 7 > 1.00 AND 5 > 1.00
# Precision: 0.95
# Coverage: 0.11
# quartile
# Anchor: 6 > 7.00 AND 8 > 0.00 AND 10 <= 1.00 AND 11 <= 0.00 AND 1 <= 2.00 AND 0 > 11.00 AND 12 > 3.00
# Precision: 0.94
# Coverage: 0.00
# quartile
# Anchor: 10 > 1.00 AND 0.00 < 5 <= 1.00 AND 0.00 < 7 <= 1.00 AND 9 <= 0.00 AND 2.00 < 4 <= 5.00 AND 3 <= 5.00 AND 0 <= 11.00 AND 2 <= 8661.00 AND 8 > 0.00 AND 12 > 3.00 AND 13 <= 0.00
# Precision: 0.72
# Coverage: 0.00
# quartile
# Anchor: 6 > 7.00 AND 8 > 0.00 AND 10 <= 1.00 AND 11 <= 0.00 AND 4 > 5.00
# Precision: 0.99
# Coverage: 0.01
# quartile
# Anchor: 13 > 0.00 AND 10 <= 1.00 AND 8 > 0.00 AND 4 <= 1.00 AND 12 <= 0.00 AND 1 <= 2.00 AND 11 <= 0.00
# Precision: 0.95
# Coverage: 0.01
# quartile
# Anchor: 7 > 1.00 AND 6 > 7.00
# Precision: 0.95
# Coverage: 0.07
# quartile
# Anchor: 6 > 4.00 AND 13 > 0.00 AND 10 <= 1.00 AND 3 > 1.00 AND 11 <= 0.00
# Precision: 0.95
# Coverage: 0.03
# quartile
# Anchor: 5 <= 0.00 AND 10 <= 1.00
# Precision: 0.98
# Coverage: 0.32
# quartile
# Anchor: 0 > 34.00 AND 8 > 0.00 AND 10 <= 1.00 AND 1 <= 2.00 AND 11 <= 0.00 AND 4 <= 2.00 AND 13 <= 0.00 AND 2 <= 14408.00 AND 12 <= 3.00
# Precision: 0.96
# Coverage: 0.00
# quartile
# Anchor: 6 > 7.00 AND 8 > 0.00 AND 10 <= 1.00 AND 0 > 34.00 AND 11 <= 0.00
# Precision: 0.95
# Coverage: 0.01
# quartile
# Anchor: 6 > 7.00 AND 8 > 0.00 AND 10 <= 1.00 AND 0 > 34.00 AND 11 <= 0.00
# Precision: 0.96
# Coverage: 0.01
# quartile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.99
# Coverage: 0.19
# quartile
# Anchor: 8 > 0.00 AND 10 <= 1.00 AND 11 <= 0.00 AND 12 <= 3.00 AND 4 <= 5.00 AND 13 <= 0.00 AND 11.00 < 0 <= 22.00 AND 1.00 < 3 <= 5.00 AND 9 <= 1.00 AND 2 <= 8661.00
# Precision: 0.91
# Coverage: 0.00
# quartile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.99
# Coverage: 0.18
# quartile
# Anchor: 7 > 1.00 AND 5 > 1.00
# Precision: 0.96
# Coverage: 0.10
# quartile
# Anchor: 10 > 1.00 AND 0.00 < 7 <= 1.00 AND 0.00 < 5 <= 1.00 AND 6 <= 7.00 AND 3 <= 5.00 AND 0 <= 11.00 AND 9 <= 0.00 AND 0.00 < 12 <= 3.00 AND 4 <= 1.00 AND 1 > 2.00 AND 2 <= 8661.00
# Precision: 0.78
# Coverage: 0.00
# quartile
# Anchor: 0.00 < 5 <= 1.00 AND 0.00 < 7 <= 1.00 AND 9 <= 0.00 AND 6 <= 7.00 AND 13 <= 0.00 AND 12 <= 9.00 AND 2 <= 14408.00 AND 4 <= 5.00 AND 3 <= 5.00 AND 1 > 2.00
# Precision: 0.52
# Coverage: 0.03
# quartile
# Anchor: 7 > 1.00 AND 5 > 1.00
# Precision: 0.96
# Coverage: 0.10
# quartile
# Anchor: 5 <= 0.00
# Precision: 0.95
# Coverage: 0.33
# quartile
# Anchor: 6 > 7.00 AND 8 > 0.00 AND 10 <= 1.00 AND 11 <= 0.00 AND 12 > 9.00 AND 3 <= 2.00 AND 1 <= 2.00
# Precision: 0.95
# Coverage: 0.00
# quartile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.99
# Coverage: 0.19
# quartile
# Anchor: 7 > 3.00 AND 9 > 0.00
# Precision: 0.97
# Coverage: 0.09
# quartile
# Anchor: 5 <= 0.00
# Precision: 0.96
# Coverage: 0.33
# quartile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.99
# Coverage: 0.19
# quartile
# Anchor: 10 > 1.00 AND 0.00 < 5 <= 1.00 AND 6 <= 7.00 AND 7 > 0.00 AND 13 <= 0.00
# Precision: 0.67
# Coverage: 0.04
# quartile
# Anchor: 7 > 1.00 AND 5 > 1.00
# Precision: 0.96
# Coverage: 0.10
# quartile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.99
# Coverage: 0.19
# quartile
# Anchor: 5 <= 0.00 AND 6 > 7.00
# Precision: 0.98
# Coverage: 0.07
# quartile
# Anchor: 5 <= 0.00 AND 9 > 0.00
# Precision: 0.97
# Coverage: 0.15
# quartile
# Anchor: 0 > 34.00 AND 8 > 0.00 AND 10 <= 1.00 AND 12 <= 0.00 AND 11 <= 0.00 AND 1 <= 2.00 AND 3 <= 5.00 AND 13 <= 0.00 AND 4 <= 5.00 AND 9 <= 1.00
# Precision: 0.92
# Coverage: 0.01
# quartile
# Anchor: 6 > 7.00 AND 8 > 0.00 AND 10 <= 1.00 AND 0 > 34.00 AND 11 <= 0.00 AND 1 <= 2.00 AND 2 <= 14408.00
# Precision: 0.95
# Coverage: 0.00
# quartile
# Anchor: 5 <= 0.00
# Precision: 0.95
# Coverage: 0.33
# decile
# Anchor: 5 <= 0.00 AND 6 <= 0.00
# Precision: 0.99
# Coverage: 0.05
# decile
# Anchor: 1 <= 1.00 AND 8 > 0.00 AND 10 <= 1.00 AND 0.00 < 3 <= 2.00
# Precision: 0.96
# Coverage: 0.00
# decile
# Anchor: 1 <= 1.00 AND 8 > 0.00 AND 10 <= 1.00 AND 3 > 0.00 AND 4 <= 2.00
# Precision: 0.96
# Coverage: 0.00
# decile
# Anchor: 9 <= 1.00 AND 0.00 < 5 <= 1.00 AND 6 <= 3.00
# Precision: 0.59
# Coverage: 0.18
# decile
# Anchor: 5 <= 0.00 AND 10 <= 1.00
# Precision: 0.98
# Coverage: 0.31
# decile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.98
# Coverage: 0.18
# decile
# Anchor: 5 <= 0.00 AND 7 > 2.00
# Precision: 0.98
# Coverage: 0.19
# decile
# Anchor: 3.00 < 6 <= 5.00 AND 8 > 0.00 AND 4 > 0.00 AND 11 <= 0.00
# Precision: 0.96
# Coverage: 0.03
# decile
# Anchor: 5 <= 0.00 AND 7 > 2.00
# Precision: 1.00
# Coverage: 0.18
# decile
# Anchor: 0.00 < 8 <= 1.00 AND 0.00 < 5 <= 1.00 AND 5.00 < 6 <= 6.00 AND 4.00 < 0 <= 9.00 AND 9 <= 0.00 AND 0.00 < 7 <= 1.00 AND 13 <= 0.00 AND 3.00 < 12 <= 4.00 AND 2 <= 1494.00 AND 2.00 < 4 <= 8.00 AND 2.00 < 3 <= 8.00 AND 11 <= 0.00 AND 1 <= 1.00 AND 10 <= 1.00
# Precision: 0.63
# Coverage: 0.00
# decile
# Anchor: 10 > 1.00 AND 0.00 < 5 <= 1.00 AND 6 <= 1.00 AND 0.00 < 7 <= 1.00 AND 0 <= 13.00 AND 8 <= 1.00 AND 13 <= 0.00 AND 12 <= 0.00 AND 3 <= 1.00 AND 9 <= 0.00 AND 4 <= 1.00 AND 1 <= 2.00 AND 2 <= 1494.00 AND 11 <= 0.00
# Precision: 0.80
# Coverage: 0.00
# decile
# Anchor: 6 > 6.00 AND 8 > 0.00 AND 10 <= 1.00 AND 3 > 0.00
# Precision: 0.96
# Coverage: 0.04
# decile
# Anchor: 5 <= 0.00
# Precision: 0.96
# Coverage: 0.32
# decile
# Anchor: 0.00 < 4 <= 1.00 AND 8 > 0.00 AND 10 <= 1.00 AND 11 <= 0.00
# Precision: 0.96
# Coverage: 0.05
# decile
# Anchor: 10 > 1.00 AND 0.00 < 5 <= 1.00 AND 6 <= 3.00 AND 0.00 < 7 <= 1.00 AND 12 <= 23.00 AND 0 <= 22.00 AND 9 <= 0.00 AND 8 <= 1.00 AND 2 <= 15731.00 AND 13 <= 0.00 AND 1 <= 2.00
# Precision: 0.79
# Coverage: 0.01
# decile
# Anchor: 6 > 6.00 AND 0.00 < 8 <= 1.00 AND 10 <= 1.00 AND 0 <= 27.00
# Precision: 0.96
# Coverage: 0.02
# decile
# Anchor: 5 <= 0.00 AND 10 <= 1.00
# Precision: 0.98
# Coverage: 0.32
# decile
# Anchor: 7 > 3.00 AND 6 > 6.00
# Precision: 0.98
# Coverage: 0.03
# decile
# Anchor: 7 > 2.00 AND 9 > 0.00
# Precision: 0.98
# Coverage: 0.16
# decile
# Anchor: 3 > 8.00 AND 6 > 3.00 AND 10 <= 1.00 AND 8 > 0.00
# Precision: 0.97
# Coverage: 0.01
# decile
# Anchor: 0.00 < 8 <= 1.00 AND 0.00 < 5 <= 1.00 AND 4.00 < 0 <= 9.00 AND 0.00 < 7 <= 1.00 AND 8.00 < 12 <= 12.00 AND 8.00 < 6 <= 10.00 AND 9 <= 0.00 AND 5.00 < 4 <= 8.00 AND 13 <= 0.00 AND 2 <= 1494.00 AND 5.00 < 3 <= 8.00 AND 1 <= 1.00 AND 11 <= 0.00 AND 10 <= 1.00
# Precision: 0.65
# Coverage: 0.00
# decile
# Anchor: 5 <= 0.00
# Precision: 0.95
# Coverage: 0.32
# decile
# Anchor: 7 > 1.00 AND 5 > 1.00 AND 9 > 0.00
# Precision: 0.97
# Coverage: 0.07
# decile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.98
# Coverage: 0.18
# decile
# Anchor: 0.00 < 7 <= 1.00 AND 0.00 < 5 <= 1.00 AND 13 <= 0.00 AND 4.00 < 6 <= 5.00 AND 0.00 < 8 <= 1.00 AND 9 <= 0.00 AND 12 <= 0.00 AND 0 <= 4.00 AND 2 <= 1494.00 AND 5.00 < 4 <= 8.00 AND 5.00 < 3 <= 8.00 AND 11 <= 0.00 AND 1 <= 1.00 AND 10 <= 1.00
# Precision: 0.64
# Coverage: 0.00
# decile
# Anchor: 6 > 5.00 AND 8 > 0.00 AND 0.00 < 3 <= 2.00 AND 11 <= 0.00 AND 0 <= 32.00
# Precision: 0.96
# Coverage: 0.02
# decile
# Anchor: 5 <= 0.00 AND 6 > 5.00
# Precision: 0.98
# Coverage: 0.10
# decile
# Anchor: 7 > 2.00 AND 10 <= 1.00
# Precision: 0.96
# Coverage: 0.28
# decile
# Anchor: 6 > 5.00 AND 8 > 0.00 AND 10 <= 1.00 AND 12 > 4.00 AND 0.00 < 4 <= 2.00
# Precision: 0.97
# Coverage: 0.01
# decile
# Anchor: 7 > 1.00 AND 5 <= 0.00
# Precision: 0.99
# Coverage: 0.18
# decile
# Anchor: 0.00 < 7 <= 2.00 AND 0.00 < 5 <= 1.00 AND 12 > 0.00 AND 0 <= 37.00 AND 6 <= 8.00
# Precision: 0.55
# Coverage: 0.17
# decile
# Anchor: 7 > 2.00 AND 9 > 0.00
# Precision: 0.98
# Coverage: 0.17
# decile
# Anchor: 7 > 1.00 AND 5 <= 0.00
# Precision: 0.99
# Coverage: 0.19
# decile
# Anchor: 1 <= 1.00 AND 8 > 0.00 AND 10 <= 1.00 AND 0.00 < 3 <= 5.00 AND 9.00 < 0 <= 27.00 AND 11 <= 0.00
# Precision: 0.95
# Coverage: 0.00
# decile
# Anchor: 7 > 2.00 AND 6 > 6.00
# Precision: 0.97
# Coverage: 0.07
# decile
# Anchor: 0.00 < 4 <= 2.00 AND 8 > 0.00 AND 13 <= 0.00 AND 10 <= 1.00
# Precision: 0.97
# Coverage: 0.04
# decile
# Anchor: 0.00 < 7 <= 2.00 AND 0.00 < 5 <= 1.00 AND 6 <= 5.00 AND 0 <= 22.00 AND 9 <= 1.00
# Precision: 0.58
# Coverage: 0.14
# decile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.99
# Coverage: 0.18
# decile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.99
# Coverage: 0.18
# decile
# Anchor: 5 <= 0.00 AND 6 > 5.00
# Precision: 0.98
# Coverage: 0.10
# decile
# Anchor: 1 <= 1.00 AND 6 > 3.00 AND 10 <= 1.00 AND 8 > 0.00 AND 3 > 5.00
# Precision: 0.96
# Coverage: 0.00
# decile
# Anchor: 6 > 8.00 AND 0.00 < 8 <= 1.00 AND 4 > 0.00 AND 10 <= 1.00
# Precision: 0.96
# Coverage: 0.02
# decile
# Anchor: 1 <= 1.00 AND 8 > 0.00 AND 10 <= 1.00 AND 0.00 < 4 <= 2.00
# Precision: 0.97
# Coverage: 0.00
# decile
# Anchor: 1 <= 1.00 AND 3.00 < 6 <= 4.00
# Precision: 0.97
# Coverage: 0.01
# decile
# Anchor: 5 <= 0.00 AND 10 <= 1.00
# Precision: 0.98
# Coverage: 0.32
# decile
# Anchor: 7 > 2.00 AND 6 > 3.00
# Precision: 0.98
# Coverage: 0.18
# decile
# Anchor: 7 <= 4.00 AND 0.00 < 5 <= 1.00 AND 6 <= 6.00 AND 8 <= 1.00 AND 12 <= 23.00
# Precision: 0.52
# Coverage: 0.30
# decile
# Anchor: 10 > 1.00 AND 0.00 < 5 <= 1.00 AND 0.00 < 7 <= 1.00 AND 4 <= 5.00 AND 0.00 < 12 <= 4.00 AND 13 <= 0.00 AND 9 <= 0.00 AND 3 <= 5.00 AND 2 <= 3065.00 AND 22.00 < 0 <= 48.00 AND 1 <= 2.00 AND 8 <= 1.00
# Precision: 0.75
# Coverage: 0.00
# decile
# Anchor: 13 <= 2.00 AND 3 <= 0.00 AND 5 > 0.00 AND 4 <= 0.00 AND 1.00 < 7 <= 2.00 AND 2 <= 1494.00 AND 8 <= 0.00 AND 6 > 10.00 AND 27.00 < 0 <= 37.00 AND 12 <= 23.00 AND 9 > 0.00 AND 1 <= 1.00
# Precision: 0.99
# Coverage: 0.00
# decile
# Anchor: 7 > 1.00 AND 5 > 1.00
# Precision: 0.96
# Coverage: 0.10
# decile
# Anchor: 5 <= 0.00 AND 8 > 0.00
# Precision: 0.99
# Coverage: 0.06
# decile
# Anchor: 12 <= 23.00 AND 5 > 0.00 AND 9 <= 0.00 AND 2 <= 3065.00
# Precision: 0.43
# Coverage: 0.10
# decile
# Anchor: 5 <= 0.00 AND 10 <= 1.00
# Precision: 0.98
# Coverage: 0.31
# decile
# Anchor: 7 > 1.00 AND 5 <= 0.00
# Precision: 0.99
# Coverage: 0.18
# decile
# Anchor: 7 > 1.00 AND 6 > 6.00
# Precision: 0.96
# Coverage: 0.08
# decile
# Anchor: 5 <= 0.00 AND 6 > 6.00
# Precision: 0.99
# Coverage: 0.08
# decile
# Anchor: 8 <= 1.00 AND 0.00 < 5 <= 1.00 AND 0 <= 37.00 AND 13 <= 0.00 AND 0.00 < 7 <= 3.00
# Precision: 0.48
# Coverage: 0.32
# decile
# Anchor: 10 > 1.00 AND 0.00 < 5 <= 1.00 AND 0.00 < 7 <= 1.00 AND 0.00 < 6 <= 8.00 AND 9 <= 0.00 AND 13 <= 0.00 AND 0 <= 9.00 AND 4 <= 8.00 AND 0.00 < 12 <= 8.00 AND 8 <= 1.00 AND 3 <= 8.00 AND 2 <= 15731.00
# Precision: 0.74
# Coverage: 0.00
# decile
# Anchor: 10 > 1.00 AND 0.00 < 5 <= 1.00 AND 3 <= 1.00 AND 6 <= 8.00 AND 0.00 < 7 <= 1.00 AND 1 > 4.00 AND 4 <= 1.00 AND 9 <= 0.00 AND 2 <= 6674.00 AND 0.00 < 12 <= 23.00 AND 8 <= 1.00 AND 13 <= 0.00 AND 0 <= 32.00
# Precision: 0.79
# Coverage: 0.00
# decile
# Anchor: 6 > 6.00 AND 8 > 0.00 AND 10 <= 1.00 AND 0.00 < 3 <= 2.00
# Precision: 0.97
# Coverage: 0.02
# decile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.99
# Coverage: 0.19
# decile
# Anchor: 13 <= 0.00 AND 5 > 0.00 AND 9 <= 0.00 AND 0.00 < 7 <= 2.00 AND 3 > 2.00 AND 6 <= 10.00 AND 0 <= 37.00
# Precision: 0.49
# Coverage: 0.12
# decile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.99
# Coverage: 0.18
# decile
# Anchor: 5 <= 0.00 AND 6 <= 0.00
# Precision: 1.00
# Coverage: 0.05
# decile
# Anchor: 7 > 1.00 AND 5 > 1.00 AND 6 > 6.00
# Precision: 0.97
# Coverage: 0.02
# decile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.98
# Coverage: 0.19
# decile
# Anchor: 10 > 1.00 AND 0.00 < 5 <= 1.00 AND 4.00 < 6 <= 6.00 AND 0.00 < 7 <= 1.00 AND 13 <= 0.00 AND 32.00 < 0 <= 37.00 AND 8 <= 1.00 AND 12.00 < 12 <= 23.00 AND 9 <= 0.00 AND 1 > 4.00 AND 1.00 < 4 <= 5.00 AND 3 <= 2.00 AND 2 <= 6674.00 AND 11 <= 0.00
# Precision: 0.78
# Coverage: 0.00
# decile
# Anchor: 5 <= 0.00 AND 6 > 3.00
# Precision: 0.98
# Coverage: 0.19
# decile
# Anchor: 7 > 1.00 AND 5 > 2.00
# Precision: 0.97
# Coverage: 0.04
# decile
# Anchor: 6 > 6.00 AND 8 > 0.00 AND 10 <= 1.00 AND 3 > 0.00 AND 4 <= 2.00
# Precision: 0.96
# Coverage: 0.02
# decile
# Anchor: 10 > 1.00 AND 0.00 < 5 <= 1.00 AND 0.00 < 6 <= 8.00 AND 13 <= 0.00 AND 0.00 < 7 <= 1.00 AND 8 <= 1.00 AND 12 <= 12.00 AND 4 <= 5.00 AND 3 <= 5.00 AND 9 <= 0.00 AND 2 <= 6674.00 AND 4.00 < 0 <= 37.00
# Precision: 0.74
# Coverage: 0.01
# decile
# Anchor: 6 > 6.00 AND 8 > 0.00 AND 10 <= 1.00 AND 3 > 0.00
# Precision: 0.95
# Coverage: 0.04
# decile
# Anchor: 1 <= 1.00 AND 6 <= 0.00 AND 13 > 0.00 AND 8 > 0.00 AND 10 <= 1.00
# Precision: 0.97
# Coverage: 0.00
# decile
# Anchor: 7 > 1.00 AND 6 > 6.00
# Precision: 0.95
# Coverage: 0.08
# decile
# Anchor: 6 > 5.00 AND 8 > 0.00 AND 10 <= 1.00 AND 4 > 0.00 AND 13 > 2.00 AND 11 <= 0.00
# Precision: 0.97
# Coverage: 0.01
# decile
# Anchor: 5 <= 0.00 AND 10 <= 1.00
# Precision: 0.97
# Coverage: 0.32
# decile
# Anchor: 6 <= 0.00 AND 8 > 0.00 AND 0.00 < 4 <= 1.00 AND 10 <= 1.00
# Precision: 0.95
# Coverage: 0.01
# decile
# Anchor: 6 > 6.00 AND 8 > 0.00 AND 3 > 0.00 AND 10 <= 1.00
# Precision: 0.95
# Coverage: 0.03
# decile
# Anchor: 6 > 6.00 AND 8 > 0.00 AND 10 <= 1.00 AND 3 > 0.00 AND 4 <= 2.00
# Precision: 0.97
# Coverage: 0.02
# decile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.99
# Coverage: 0.18
# decile
# Anchor: 6 <= 0.00 AND 8 > 0.00 AND 3 > 0.00 AND 10 <= 1.00 AND 11 <= 0.00 AND 17.00 < 0 <= 27.00
# Precision: 0.95
# Coverage: 0.00
# decile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.99
# Coverage: 0.17
# decile
# Anchor: 7 > 1.00 AND 5 > 1.00
# Precision: 0.95
# Coverage: 0.10
# decile
# Anchor: 10 > 1.00 AND 0.00 < 5 <= 1.00 AND 4.00 < 6 <= 6.00 AND 9 <= 0.00 AND 0.00 < 7 <= 1.00 AND 13 <= 2.00 AND 0.00 < 12 <= 8.00 AND 8 <= 1.00 AND 0 <= 32.00 AND 3 <= 1.00 AND 4 <= 1.00 AND 1 > 4.00 AND 2 <= 3065.00
# Precision: 0.79
# Coverage: 0.00
# decile
# Anchor: 0.00 < 7 <= 1.00 AND 0.00 < 5 <= 1.00 AND 0.00 < 6 <= 6.00 AND 0.00 < 12 <= 4.00 AND 9 <= 0.00 AND 0 <= 48.00 AND 2 <= 1494.00 AND 13 <= 0.00 AND 8 <= 1.00 AND 2.00 < 1 <= 4.00 AND 2.00 < 4 <= 5.00 AND 3 <= 5.00
# Precision: 0.50
# Coverage: 0.00
# decile
# Anchor: 7 > 1.00 AND 5 > 1.00 AND 9 > 0.00
# Precision: 0.97
# Coverage: 0.07
# decile
# Anchor: 5 <= 0.00 AND 10 <= 1.00
# Precision: 0.97
# Coverage: 0.31
# decile
# Anchor: 6 > 6.00 AND 8 > 0.00 AND 3 > 0.00 AND 10 <= 1.00
# Precision: 0.95
# Coverage: 0.04
# decile
# Anchor: 5 <= 0.00
# Precision: 0.95
# Coverage: 0.33
# decile
# Anchor: 7 > 2.00 AND 9 > 0.00
# Precision: 0.97
# Coverage: 0.16
# decile
# Anchor: 5 <= 0.00 AND 10 <= 1.00
# Precision: 0.98
# Coverage: 0.31
# decile
# Anchor: 5 <= 0.00
# Precision: 0.95
# Coverage: 0.33
# decile
# Anchor: 10 > 1.00 AND 0.00 < 5 <= 1.00 AND 0.00 < 6 <= 5.00 AND 3.00 < 7 <= 4.00 AND 13 <= 0.00 AND 8 <= 1.00 AND 2 <= 3065.00 AND 9 <= 1.00 AND 3.00 < 12 <= 4.00 AND 5.00 < 3 <= 8.00 AND 5.00 < 4 <= 8.00 AND 1 <= 1.00 AND 0 <= 32.00 AND 11 <= 0.00
# Precision: 0.72
# Coverage: 0.00
# decile
# Anchor: 7 > 1.00 AND 6 > 6.00
# Precision: 0.96
# Coverage: 0.08
# decile
# Anchor: 5 <= 0.00 AND 7 > 1.00
# Precision: 0.99
# Coverage: 0.18
# decile
# Anchor: 5 <= 0.00 AND 10 <= 1.00
# Precision: 0.98
# Coverage: 0.32
# decile
# Anchor: 5 <= 0.00 AND 9 > 0.00
# Precision: 0.98
# Coverage: 0.14
# decile
# Anchor: 1 <= 1.00 AND 0.00 < 8 <= 1.00 AND 10 <= 1.00 AND 4 > 0.00 AND 11 <= 0.00
# Precision: 0.96
# Coverage: 0.01
# decile
# Anchor: 6 > 6.00 AND 10 <= 1.00 AND 0 > 37.00 AND 8 > 0.00 AND 11 <= 0.00
# Precision: 0.96
# Coverage: 0.01
# decile
# Anchor: 7 > 1.00 AND 5 <= 0.00
# Precision: 0.98
# Coverage: 0.18
# entropy
# Anchor: 5 <= 0.50 AND 6 <= 0.50
# Precision: 1.00
# Coverage: 0.05
# entropy
# Anchor: 8 > 0.50 AND 10 <= 0.50 AND 0.50 < 3 <= 2.50 AND 4 > 0.50 AND 6 <= 4.50
# Precision: 0.97
# Coverage: 0.00
# entropy
# Anchor: 8 > 0.50 AND 4 > 0.50 AND 1.50 < 3 <= 2.50
# Precision: 0.97
# Coverage: 0.01
# entropy
# Anchor: 0.50 < 10 <= 55.50 AND 0.50 < 7 <= 2.50 AND 0.50 < 5 <= 1.50 AND 6 <= 1.50 AND 8 <= 0.50 AND 0.50 < 0 <= 3.50 AND 2.50 < 3 <= 3.50 AND 2.50 < 4 <= 3.50 AND 2 <= 1775.50 AND 13 <= 2.50
# Precision: 0.96
# Coverage: 0.00
# entropy
# Anchor: 5 <= 0.50 AND 10 <= 0.50
# Precision: 0.99
# Coverage: 0.00
# entropy
# Anchor: 5 <= 0.50 AND 7 > 1.50
# Precision: 0.98
# Coverage: 0.18
# entropy
# Anchor: 7 > 2.50 AND 5 <= 0.50
# Precision: 0.99
# Coverage: 0.19
# entropy
# Anchor: 6 > 3.50 AND 0.50 < 8 <= 1.50 AND 10 <= 5.50 AND 11 <= 2.50 AND 3 > 0.50
# Precision: 0.96
# Coverage: 0.06
# entropy
# Anchor: 5 <= 0.50 AND 7 > 2.50
# Precision: 0.99
# Coverage: 0.18
# entropy
# Anchor: 13 <= 4.50 AND 0.50 < 5 <= 1.50 AND 6 <= 6.50 AND 12 > 3.50 AND 7 <= 2.50 AND 8 <= 2.50 AND 2 <= 21513.50
# Precision: 0.55
# Coverage: 0.14
# entropy
# Anchor: 1.50 < 10 <= 12.50 AND 0.50 < 5 <= 1.50 AND 0.50 < 7 <= 1.50 AND 6 <= 1.50 AND 0 <= 11.50 AND 3 <= 3.50
# Precision: 0.95
# Coverage: 0.00
# entropy
# Anchor: 6 > 6.50 AND 8 > 0.50 AND 10 <= 1.50 AND 3 > 0.50 AND 11 <= 0.50
# Precision: 0.96
# Coverage: 0.03
# entropy
# Anchor: 5 <= 0.50 AND 7 > 1.50
# Precision: 0.99
# Coverage: 0.18
# entropy
# Anchor: 8 > 0.50 AND 10 <= 0.50 AND 0.50 < 4 <= 2.50 AND 1 <= 2.50
# Precision: 0.96
# Coverage: 0.00
# entropy
# Anchor: 1.50 < 10 <= 12.50 AND 0.50 < 5 <= 1.50 AND 0.50 < 7 <= 2.50 AND 6 <= 1.50 AND 2 <= 833.50 AND 1 <= 4.50 AND 13 <= 2.50 AND 4 <= 2.50 AND 3 <= 3.50
# Precision: 0.95
# Coverage: 0.00
# entropy
# Anchor: 6 > 6.50 AND 8 > 0.50 AND 10 <= 1.50 AND 3 > 3.50 AND 11 <= 2.50
# Precision: 0.96
# Coverage: 0.02
# entropy
# Anchor: 5 <= 0.50 AND 10 <= 0.50
# Precision: 0.98
# Coverage: 0.00
# entropy
# Anchor: 7 > 2.50 AND 10 <= 1.50
# Precision: 0.97
# Coverage: 0.27
# entropy
# Anchor: 7 > 2.50 AND 9 > 0.50
# Precision: 0.97
# Coverage: 0.17
# entropy
# Anchor: 6 > 3.50 AND 8 > 0.50 AND 10 <= 1.50 AND 4 > 0.50 AND 12 > 5.50
# Precision: 0.95
# Coverage: 0.03
# entropy
# Anchor: 0 > 0.50 AND 0.50 < 5 <= 1.50 AND 12 > 2.50 AND 9 <= 0.50 AND 7 <= 1.50 AND 13 <= 1.50 AND 2 <= 21513.50 AND 1 <= 6.50 AND 8 <= 1.50 AND 10 <= 55.50 AND 6 > 0.50
# Precision: 0.52
# Coverage: 0.17
# entropy
# Anchor: 5 <= 0.50 AND 10 <= 0.50
# Precision: 0.99
# Coverage: 0.00
# entropy
# Anchor: 7 > 1.50 AND 10 <= 1.50 AND 5 > 1.50
# Precision: 0.96
# Coverage: 0.09
# entropy
# Anchor: 5 <= 0.50 AND 10 <= 1.50
# Precision: 0.98
# Coverage: 0.32
# entropy
# Anchor: 0.50 < 5 <= 1.50 AND 0.50 < 7 <= 1.50 AND 4.50 < 6 <= 6.50 AND 9 <= 0.50 AND 13 <= 1.50 AND 1 <= 2.50 AND 11 <= 0.50 AND 2 <= 833.50 AND 8 <= 1.50 AND 10 <= 0.50 AND 12 <= 5.50 AND 0 <= 0.50 AND 3 > 0.50
# Precision: 0.51
# Coverage: 0.00
# entropy
# Anchor: 6 > 3.50 AND 8 > 0.50 AND 10 <= 5.50 AND 4 > 0.50 AND 3 <= 2.50 AND 11 <= 0.50
# Precision: 0.96
# Coverage: 0.04
# entropy
# Anchor: 5 <= 0.50
# Precision: 0.95
# Coverage: 0.33
# entropy
# Anchor: 7 > 2.50 AND 10 <= 1.50
# Precision: 0.97
# Coverage: 0.28
# entropy
# Anchor: 6 > 3.50 AND 8 > 0.50 AND 0.50 < 4 <= 2.50 AND 0 > 9.50
# Precision: 0.95
# Coverage: 0.03
# entropy
# Anchor: 5 <= 0.50 AND 7 > 1.50
# Precision: 0.98
# Coverage: 0.19
# entropy
# Anchor: 4.50 < 1 <= 5.50 AND 0.50 < 5 <= 1.50 AND 12 > 3.50 AND 3 <= 3.50 AND 0.50 < 7 <= 1.50 AND 2.50 < 4 <= 3.50 AND 13 <= 1.50 AND 2 <= 833.50 AND 0.50 < 10 <= 12.50 AND 8 <= 0.50 AND 11 <= 12.50 AND 6 > 4.50 AND 9 <= 0.50 AND 11.50 < 0 <= 27.50
# Precision: 0.83
# Coverage: 0.00
# entropy
# Anchor: 7 > 2.50 AND 9 > 0.50
# Precision: 0.97
# Coverage: 0.16
# entropy
# Anchor: 5 <= 0.50 AND 10 <= 0.50
# Precision: 0.99
# Coverage: 0.00
# entropy
# Anchor: 8 > 0.50 AND 3 > 0.50 AND 10 <= 1.50 AND 12 <= 1.50 AND 0.50 < 4 <= 3.50 AND 1 <= 2.50 AND 11 <= 2.50
# Precision: 0.95
# Coverage: 0.02
# entropy
# Anchor: 7 > 2.50 AND 6 > 6.50
# Precision: 0.98
# Coverage: 0.08
# entropy
# Anchor: 8 > 0.50 AND 3 > 0.50 AND 4 <= 2.50 AND 10 <= 1.50 AND 12 <= 1.50
# Precision: 0.96
# Coverage: 0.03
# entropy
# Anchor: 13 <= 5.50 AND 0.50 < 5 <= 1.50 AND 6 <= 6.50 AND 7 <= 2.50 AND 0 <= 11.50
# Precision: 0.51
# Coverage: 0.10
# entropy
# Anchor: 5 <= 0.50 AND 7 > 1.50
# Precision: 0.99
# Coverage: 0.19
# entropy
# Anchor: 5 <= 0.50 AND 7 > 1.50
# Precision: 0.99
# Coverage: 0.18
# entropy
# Anchor: 5 <= 0.50
# Precision: 0.97
# Coverage: 0.33
# entropy
# Anchor: 6 > 3.50 AND 8 > 0.50 AND 10 <= 1.50 AND 4 > 0.50 AND 9.50 < 0 <= 27.50
# Precision: 0.96
# Coverage: 0.03
# entropy
# Anchor: 6 > 6.50 AND 8 > 0.50 AND 10 <= 1.50 AND 4 > 0.50 AND 11 <= 2.50
# Precision: 0.95
# Coverage: 0.04
# entropy
# Anchor: 0.50 < 4 <= 2.50 AND 8 > 0.50 AND 10 <= 1.50 AND 11 <= 2.50 AND 12 <= 2.50
# Precision: 0.95
# Coverage: 0.03
# entropy
# Anchor: 4 <= 1.50 AND 8 > 0.50 AND 3.50 < 6 <= 4.50
# Precision: 0.95
# Coverage: 0.01
# entropy
# Anchor: 5 <= 0.50 AND 10 <= 1.50
# Precision: 0.98
# Coverage: 0.32
# entropy
# Anchor: 7 > 3.50 AND 9 > 0.50
# Precision: 0.97
# Coverage: 0.10
# entropy
# Anchor: 0.50 < 5 <= 1.50 AND 0.50 < 6 <= 6.50 AND 9 <= 0.50 AND 4 > 2.50 AND 13 <= 4.50 AND 3 > 2.50 AND 7 <= 2.50 AND 10 <= 55.50
# Precision: 0.57
# Coverage: 0.11
# entropy
# Anchor: 1.50 < 10 <= 12.50 AND 0.50 < 5 <= 1.50 AND 0.50 < 7 <= 1.50 AND 4 <= 3.50 AND 0.50 < 12 <= 5.50 AND 13 <= 1.50 AND 0 > 10.50 AND 6 > 6.50 AND 9 <= 0.50
# Precision: 0.95
# Coverage: 0.00
# entropy
# Anchor: 8 <= 0.50 AND 4 <= 0.50 AND 1.50 < 7 <= 2.50 AND 5 > 0.50 AND 3 <= 0.50 AND 2 <= 833.50 AND 13 <= 1.50 AND 6 > 6.50 AND 0 > 11.50 AND 1 <= 2.50 AND 10 <= 0.50 AND 11 <= 0.50 AND 9 > 0.50
# Precision: 0.76
# Coverage: 0.00
# entropy
# Anchor: 7 > 1.50 AND 6 > 6.50
# Precision: 0.95
# Coverage: 0.08
# entropy
# Anchor: 5 <= 0.50 AND 10 <= 1.50
# Precision: 0.98
# Coverage: 0.32
# entropy
# Anchor: 10 <= 55.50 AND 5 > 0.50 AND 9 <= 0.50 AND 7 <= 1.50 AND 3 <= 3.50 AND 13 <= 1.50 AND 6 > 0.50 AND 8 <= 1.50 AND 11 <= 12.50 AND 2 <= 21513.50 AND 4 <= 3.50 AND 1 <= 6.50
# Precision: 0.44
# Coverage: 0.23
# entropy
# Anchor: 5 <= 0.50 AND 10 <= 0.50
# Precision: 0.99
# Coverage: 0.00
# entropy
# Anchor: 5 <= 0.50 AND 7 > 1.50
# Precision: 0.98
# Coverage: 0.18
# entropy
# Anchor: 7 > 1.50 AND 5 > 2.50
# Precision: 0.95
# Coverage: 0.04
# entropy
# Anchor: 5 <= 0.50 AND 6 > 6.50
# Precision: 0.98
# Coverage: 0.08
# entropy
# Anchor: 11 <= 5.50 AND 0.50 < 5 <= 1.50 AND 6 <= 6.50 AND 2 <= 11003.50 AND 13 <= 4.50 AND 9 <= 0.50 AND 8 <= 2.50
# Precision: 0.55
# Coverage: 0.17
# entropy
# Anchor: 1.50 < 10 <= 12.50 AND 0.50 < 5 <= 1.50 AND 0.50 < 7 <= 1.50 AND 13 <= 1.50 AND 1 <= 4.50 AND 9 <= 0.50 AND 0.50 < 8 <= 1.50 AND 0.50 < 12 <= 5.50 AND 4.50 < 0 <= 10.50 AND 2 <= 21513.50 AND 11 <= 16.50
# Precision: 0.92
# Coverage: 0.00
# entropy
# Anchor: 5.50 < 10 <= 12.50 AND 1 > 5.50 AND 0.50 < 5 <= 1.50
# Precision: 0.96
# Coverage: 0.00
# entropy
# Anchor: 6 > 6.50 AND 8 > 0.50 AND 10 <= 1.50 AND 4 > 0.50 AND 3 <= 2.50
# Precision: 0.96
# Coverage: 0.02
# entropy
# Anchor: 5 <= 0.50 AND 7 > 1.50
# Precision: 0.99
# Coverage: 0.19
# entropy
# Anchor: 0.50 < 7 <= 2.50 AND 5 > 0.50 AND 9 <= 0.50 AND 2 <= 833.50 AND 13 <= 1.50 AND 10 <= 0.50 AND 2.50 < 3 <= 3.50 AND 0.50 < 8 <= 1.50 AND 2.50 < 4 <= 3.50 AND 6 > 4.50 AND 1 <= 2.50 AND 12 <= 3.50 AND 11 <= 16.50
# Precision: 0.73
# Coverage: 0.00
# entropy
# Anchor: 5 <= 0.50 AND 10 <= 0.50
# Precision: 0.99
# Coverage: 0.00
# entropy
# Anchor: 5 <= 0.50 AND 9 > 0.50
# Precision: 0.98
# Coverage: 0.14
# entropy
# Anchor: 7 > 1.50 AND 6 > 6.50 AND 5 > 1.50
# Precision: 0.98
# Coverage: 0.02
# entropy
# Anchor: 5 <= 0.50 AND 10 <= 0.50
# Precision: 0.98
# Coverage: 0.00
# entropy
# Anchor: 5.50 < 10 <= 12.50 AND 1 > 5.50 AND 0.50 < 5 <= 1.50 AND 7 <= 2.50 AND 4 <= 3.50
# Precision: 0.97
# Coverage: 0.00
# entropy
# Anchor: 5 <= 0.50 AND 10 <= 0.50
# Precision: 0.98
# Coverage: 0.00
# entropy
# Anchor: 7 > 1.50 AND 6 > 6.50
# Precision: 0.96
# Coverage: 0.08
# entropy
# Anchor: 6 > 6.50 AND 8 > 0.50 AND 10 <= 1.50 AND 3 > 0.50
# Precision: 0.95
# Coverage: 0.03
# entropy
# Anchor: 5.50 < 10 <= 12.50 AND 0.50 < 5 <= 1.50 AND 0.50 < 7 <= 1.50 AND 3 <= 3.50
# Precision: 0.97
# Coverage: 0.02
# entropy
# Anchor: 6 > 6.50 AND 0.50 < 8 <= 1.50 AND 10 <= 1.50 AND 3 > 0.50
# Precision: 0.96
# Coverage: 0.03
# entropy
# Anchor: 13 > 2.50 AND 6 <= 0.50 AND 10 <= 1.50 AND 0.50 < 3 <= 2.50 AND 8 > 0.50
# Precision: 1.00
# Coverage: 0.00
# entropy
# Anchor: 7 > 1.50 AND 5 > 1.50 AND 6 > 6.50
# Precision: 0.98
# Coverage: 0.02
# entropy
# Anchor: 8 > 2.50 AND 6 > 3.50 AND 10 <= 1.50 AND 13 > 4.50
# Precision: 0.97
# Coverage: 0.00
# entropy
# Anchor: 5 <= 0.50
# Precision: 0.96
# Coverage: 0.33
# entropy
# Anchor: 6 <= 0.50 AND 8 > 0.50 AND 10 <= 1.50 AND 3 > 0.50 AND 4 <= 2.50
# Precision: 0.97
# Coverage: 0.01
# entropy
# Anchor: 6 > 6.50 AND 10 <= 0.50 AND 4 > 0.50 AND 0 > 9.50 AND 11 <= 2.50
# Precision: 0.95
# Coverage: 0.00
# entropy
# Anchor: 6 > 6.50 AND 10 <= 1.50 AND 0.50 < 8 <= 1.50
# Precision: 0.95
# Coverage: 0.03
# entropy
# Anchor: 5 <= 0.50 AND 6 > 6.50
# Precision: 0.99
# Coverage: 0.08
# entropy
# Anchor: 6 <= 0.50 AND 8 > 0.50 AND 0.50 < 3 <= 3.50 AND 10 <= 1.50 AND 11 <= 2.50 AND 0 <= 27.50
# Precision: 0.96
# Coverage: 0.00
# entropy
# Anchor: 5 <= 0.50 AND 10 <= 0.50
# Precision: 0.99
# Coverage: 0.00
# entropy
# Anchor: 7 > 1.50 AND 5 > 2.50 AND 6 > 3.50
# Precision: 0.98
# Coverage: 0.03
# entropy
# Anchor: 5.50 < 10 <= 12.50 AND 1 > 5.50 AND 0.50 < 5 <= 1.50 AND 7 <= 2.50 AND 3 <= 1.50
# Precision: 0.98
# Coverage: 0.00
# entropy
# Anchor: 0.50 < 5 <= 1.50 AND 0.50 < 7 <= 1.50 AND 4.50 < 6 <= 6.50 AND 13 <= 1.50 AND 0 > 11.50 AND 10 <= 0.50 AND 2.50 < 1 <= 4.50 AND 11 <= 0.50 AND 2 <= 833.50 AND 9 <= 0.50 AND 0.50 < 8 <= 1.50 AND 0.50 < 12 <= 2.50 AND 2.50 < 4 <= 3.50 AND 2.50 < 3 <= 3.50
# Precision: 0.81
# Coverage: 0.00
# entropy
# Anchor: 7 > 1.50 AND 8 > 0.50 AND 10 <= 1.50
# Precision: 0.97
# Coverage: 0.07
# entropy
# Anchor: 5 <= 0.50 AND 10 <= 1.50
# Precision: 0.98
# Coverage: 0.31
# entropy
# Anchor: 6 > 6.50 AND 8 > 0.50 AND 10 <= 1.50 AND 3 > 0.50 AND 4 <= 2.50
# Precision: 0.97
# Coverage: 0.02
# entropy
# Anchor: 5 <= 0.50 AND 10 <= 0.50
# Precision: 0.99
# Coverage: 0.00
# entropy
# Anchor: 7 > 3.50 AND 9 > 0.50
# Precision: 0.97
# Coverage: 0.10
# entropy
# Anchor: 5 <= 0.50 AND 7 > 1.50
# Precision: 0.99
# Coverage: 0.18
# entropy
# Anchor: 5 <= 0.50
# Precision: 0.95
# Coverage: 0.32
# entropy
# Anchor: 5.50 < 10 <= 12.50 AND 0.50 < 5 <= 1.50 AND 0.50 < 7 <= 4.50 AND 13 <= 1.50 AND 8 <= 1.50 AND 11 <= 5.50 AND 0 > 0.50 AND 12 <= 5.50 AND 6 > 0.50 AND 2 <= 11013.50 AND 1 <= 6.50
# Precision: 0.91
# Coverage: 0.01
# entropy
# Anchor: 7 > 1.50 AND 5 > 2.50
# Precision: 0.96
# Coverage: 0.04
# entropy
# Anchor: 5 <= 0.50 AND 7 > 1.50
# Precision: 0.99
# Coverage: 0.17
# entropy
# Anchor: 5 <= 0.50
# Precision: 0.95
# Coverage: 0.33
# entropy
# Anchor: 5 <= 0.50
# Precision: 0.95
# Coverage: 0.33
# entropy
# Anchor: 3 > 3.50 AND 0.50 < 8 <= 1.50 AND 10 <= 1.50 AND 0 > 9.50 AND 4 > 0.50 AND 11 <= 2.50 AND 1 <= 2.50
# Precision: 0.94
# Coverage: 0.02
# entropy
# Anchor: 6 > 6.50 AND 8 > 0.50 AND 10 <= 1.50 AND 11 <= 2.50 AND 4 > 0.50 AND 3 <= 2.50
# Precision: 0.96
# Coverage: 0.02
# entropy
# Anchor: 5 <= 0.50 AND 10 <= 0.50
# Precision: 1.00
# Coverage: 0.00
# GaussianMixtureModels
# Anchor: 5 <= 1.00
# Precision: 0.95
# Coverage: 0.32
# GaussianMixtureModels
# Anchor: 1 <= 1.00 AND 10 <= 1.00
# Precision: 1.00
# Coverage: 0.00
# GaussianMixtureModels
# Anchor: 1 <= 2.00 AND 10 <= 1.00
# Precision: 1.00
# Coverage: 0.00
# GaussianMixtureModels
# Anchor: 1.00 < 7 <= 3.00 AND 0 <= 3.94 AND 1.00 < 5 <= 1.00 AND 6 <= 1.00 AND 8 <= 1.00 AND 12 <= 20.85 AND 2.11 < 4 <= 3.83 AND 2 <= 2382.11 AND 2.00 < 3 <= 3.00 AND 10 <= 40.78 AND 13 <= 0.00 AND 11 <= 18.57
# Precision: 0.95
# Coverage: 0.00
# GaussianMixtureModels
# Anchor: 5 <= 1.00 AND 10 <= 1.00
# Precision: 0.98
# Coverage: 0.00
# GaussianMixtureModels
# Anchor: 5 <= 0.00 AND 7 > 2.00
# Precision: 1.00
# Coverage: 0.19
# GaussianMixtureModels
# Anchor: 7 > 2.00
# Precision: 0.96
# Coverage: 0.30
# GaussianMixtureModels
# Anchor: 6 > 4.08 AND 8 > 1.00 AND 10 <= 5.88 AND 3 > 1.00 AND 11 <= 3.00
# Precision: 0.95
# Coverage: 0.05
# GaussianMixtureModels
# Anchor: 7 > 2.00 AND 5 <= 0.00
# Precision: 0.99
# Coverage: 0.18

In [ ]:
results_tests = {'Number': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59],
 'Name': ['MiniBatch\nKMeans',
  'Affinity\nPropagation',
  'MeanShift',
  'Spectral\nClustering',
  'Ward',
  'Agglomerative\nClustering',
  'DBSCAN',
  'OPTICS',
  'BIRCH',
  'Gaussian\nMixture',
  'MiniBatch\nKMeans',
  'Affinity\nPropagation',
  'MeanShift',
  'Spectral\nClustering',
  'Ward',
  'Agglomerative\nClustering',
  'DBSCAN',
  'OPTICS',
  'BIRCH',
  'Gaussian\nMixture',
  'MiniBatch\nKMeans',
  'Affinity\nPropagation',
  'MeanShift',
  'Spectral\nClustering',
  'Ward',
  'Agglomerative\nClustering',
  'DBSCAN',
  'OPTICS',
  'BIRCH',
  'Gaussian\nMixture',
  'MiniBatch\nKMeans',
  'Affinity\nPropagation',
  'MeanShift',
  'Spectral\nClustering',
  'Ward',
  'Agglomerative\nClustering',
  'DBSCAN',
  'OPTICS',
  'BIRCH',
  'Gaussian\nMixture',
  'MiniBatch\nKMeans',
  'Affinity\nPropagation',
  'MeanShift',
  'Spectral\nClustering',
  'Ward',
  'Agglomerative\nClustering',
  'DBSCAN',
  'OPTICS',
  'BIRCH',
  'Gaussian\nMixture',
  'MiniBatch\nKMeans',
  'Affinity\nPropagation',
  'MeanShift',
  'Spectral\nClustering',
  'Ward',
  'Agglomerative\nClustering',
  'DBSCAN',
  'OPTICS',
  'BIRCH',
  'Gaussian\nMixture'],
 'quartile mean precision': [0.9741409494291152,
  0.9767200901707267,
  0.9777231121016463,
  0.9754401523105721,
  0.9761588803018756,
  0.9721913400326241,
  0.9737389891878047,
  0.9762463037301493,
  0.9730966616969932,
  0.9756176801725835,
  0.9800150327130065,
  0.9798452063790105,
  0.9824815306394417,
  0.9821816019176953,
  0.9763242764563544,
  0.9804378863580113,
  0.9802128832260655,
  0.9814459739456329,
  0.9774114346971765,
  0.9771833474788476,
  0.9546619697430984,
  0.9513072845326669,
  0.9535273436806041,
  0.9566246926613116,
  0.9511494722811245,
  0.9520133315799743,
  0.9555835893818593,
  0.9552442675304827,
  0.9522449314463134,
  0.9539457485601555,
  0.9601432614856573,
  0.9598359227771676,
  0.9590528735426732,
  0.9613293090036144,
  0.9593817501127138,
  0.9609813353504809,
  0.9590011123074473,
  0.9617587757800141,
  0.9609325189911889,
  0.9596526820120297,
  0.9312181904471097,
  0.9304696601824612,
  0.9310669034353632,
  0.9295603511164102,
  0.9282056751806935,
  0.9285888255026977,
  0.9292644965031165,
  0.9283528792174368,
  0.9292661953915697,
  0.9296918699796757,
  0.9701401210312725,
  0.9725238555317383,
  0.9758930494068495,
  0.9742712339917426,
  0.9741989432902931,
  0.972015353216323,
  0.9709680835939047,
  0.9708920180019738,
  0.9706604763927218,
  0.9737936389012097],
 'quartile mean coverage': [0.14802966666666667,
  0.14295533333333332,
  0.14854433333333333,
  0.14917566666666668,
  0.14488466666666666,
  0.14341833333333334,
  0.13987333333333332,
  0.146675,
  0.143049,
  0.147536,
  0.19080166666666667,
  0.187856,
  0.18973066666666666,
  0.18757966666666667,
  0.19336566666666666,
  0.189676,
  0.189426,
  0.19131533333333334,
  0.190963,
  0.18968166666666667,
  0.13904833333333333,
  0.13674666666666666,
  0.13718133333333332,
  0.13763066666666668,
  0.13931233333333334,
  0.13996533333333333,
  0.13725433333333334,
  0.13499833333333333,
  0.13856966666666667,
  0.13906166666666667,
  0.268519,
  0.2765783333333333,
  0.28457333333333334,
  0.2703413333333333,
  0.278354,
  0.2798136666666667,
  0.28338033333333335,
  0.27444466666666667,
  0.2656193333333333,
  0.28033033333333335,
  0.2577756666666667,
  0.25606,
  0.2658643333333333,
  0.27398533333333336,
  0.265789,
  0.2637526666666667,
  0.25461133333333336,
  0.260033,
  0.26690566666666665,
  0.2537623333333333,
  0.14515666666666666,
  0.14283466666666667,
  0.145705,
  0.14371433333333333,
  0.144925,
  0.144486,
  0.14582233333333333,
  0.14637933333333333,
  0.14156866666666668,
  0.14545733333333333],
 'decile mean precision': [0.9971038920213745,
  0.9969264512559615,
  0.9976123131843543,
  0.9983765691436547,
  0.9969299792977103,
  0.9968626613003927,
  0.9979913169179515,
  0.9972337081937507,
  0.9973186964445969,
  0.9972098471254538,
  0.9891328983526628,
  0.9868689995644143,
  0.9862005218252159,
  0.988024197540695,
  0.9891068714673376,
  0.9876645097447353,
  0.9890950653356122,
  0.9887155395233698,
  0.9880391728582969,
  0.9899105727489956,
  0.9849668090286532,
  0.9847868908286452,
  0.9841076308389758,
  0.9856464112430593,
  0.9842926568109618,
  0.9837507034993961,
  0.984021278544243,
  0.9842305889609478,
  0.9831347626984719,
  0.985149209587626,
  0.9776374095210423,
  0.9796282059837201,
  0.9801754854551006,
  0.9787722665753902,
  0.9792607320334579,
  0.9801170058505806,
  0.9798019501837592,
  0.9795537097971566,
  0.979296019487271,
  0.9786532025731869,
  0.965523329028217,
  0.964072927057582,
  0.9646173939988656,
  0.9631738604989383,
  0.962400389293132,
  0.9617766465833505,
  0.9640819712106719,
  0.9629375578723254,
  0.9625831449399055,
  0.9651219782187156,
  0.9937823446506471,
  0.9936275182996682,
  0.9930959724208185,
  0.9952118386053295,
  0.9937347842151545,
  0.9917960821231163,
  0.9931740064928757,
  0.9928447227485795,
  0.9925713714540726,
  0.9935938700575011],
 'decile mean coverage': [0.12090133333333333,
  0.11862866666666667,
  0.115515,
  0.116382,
  0.12008233333333333,
  0.12043333333333334,
  0.11524466666666666,
  0.11993733333333334,
  0.11694766666666667,
  0.12066733333333333,
  0.171382,
  0.17408633333333334,
  0.166857,
  0.17084866666666668,
  0.174877,
  0.176585,
  0.17091433333333333,
  0.17379933333333333,
  0.17543266666666665,
  0.17572633333333335,
  0.09063933333333334,
  0.08796733333333333,
  0.09013666666666667,
  0.08900566666666666,
  0.08918,
  0.089402,
  0.09329866666666667,
  0.08477,
  0.09182166666666666,
  0.09366766666666666,
  0.20895833333333333,
  0.209115,
  0.20633166666666666,
  0.213535,
  0.215402,
  0.21611333333333332,
  0.20519766666666667,
  0.21575666666666665,
  0.205628,
  0.21117533333333333,
  0.195019,
  0.19118533333333335,
  0.19695933333333335,
  0.20262833333333333,
  0.19708633333333334,
  0.202274,
  0.201789,
  0.19372666666666666,
  0.20281966666666668,
  0.18439933333333333,
  0.135429,
  0.12891466666666668,
  0.13456733333333334,
  0.13614133333333334,
  0.14041533333333334,
  0.142954,
  0.136128,
  0.13992966666666667,
  0.13590366666666667,
  0.12999833333333333],
 'entropy mean precision': [0.5546736943463958,
  0.6831924702216334,
  0.5742985269900529,
  0.6912368579928793,
  0.6512612420016354,
  0.5577480410327924,
  0.7131512827734083,
  0.6489683278055686,
  0.5564903310737735,
  0.5507167964143963,
  0.994500264245721,
  0.9926637612502115,
  0.9919684711022512,
  0.9945480780639487,
  0.9910000337161982,
  0.9960660210619443,
  0.9954327486295268,
  0.994224991386514,
  0.9945352122223873,
  0.993219256530145,
  0.8121546110299098,
  0.9029476449977293,
  0.7668657285441015,
  0.8057825251326121,
  0.8653872990177388,
  0.7702029513375409,
  0.7378054933178303,
  0.7869030565310974,
  0.8764068320956414,
  0.7451294717396688,
  0.6404600665612533,
  0.725652894678529,
  0.7495322588696872,
  0.7622584255157223,
  0.6947365243020811,
  0.6316359138395822,
  0.7476806917716462,
  0.7286877036736926,
  0.7938951226272613,
  0.6397086623623776,
  0.7280630824701635,
  0.783295843166759,
  0.7633091986491408,
  0.8561205621282907,
  0.6944321359997999,
  0.8574966374289497,
  0.6699657553830841,
  0.767382830494807,
  0.6673124250545116,
  0.8689231240510596,
  0.7599168982715725,
  0.7216693765332052,
  0.7556427231198227,
  0.8703926726774832,
  0.7722462155785232,
  0.7387832751526346,
  0.8448530079994911,
  0.7785123573397416,
  0.88332832633553,
  0.6676585599572642],
 'entropy mean coverage': [0.900511,
  0.6291453333333333,
  0.8742583333333334,
  0.6599546666666667,
  0.7128513333333333,
  0.8971586666666667,
  0.608212,
  0.7306956666666666,
  0.8969493333333334,
  0.9176956666666667,
  0.21941733333333333,
  0.22565,
  0.22408866666666666,
  0.22039566666666666,
  0.21347933333333333,
  0.21797666666666668,
  0.218654,
  0.21922933333333333,
  0.214328,
  0.22342599999999999,
  0.439451,
  0.134746,
  0.493786,
  0.4524666666666667,
  0.2961046666666667,
  0.5287143333333333,
  0.6022206666666666,
  0.522322,
  0.25161,
  0.5304893333333334,
  0.726694,
  0.5385476666666666,
  0.4925773333333333,
  0.5321793333333333,
  0.5794673333333333,
  0.7342536666666667,
  0.5293543333333334,
  0.5207313333333333,
  0.446749,
  0.7329873333333333,
  0.46442066666666665,
  0.41477033333333335,
  0.355428,
  0.35822966666666667,
  0.497267,
  0.3541823333333333,
  0.5902326666666666,
  0.543661,
  0.697269,
  0.3431693333333333,
  0.544869,
  0.49822,
  0.5393476666666667,
  0.29434566666666667,
  0.5172746666666667,
  0.582148,
  0.41345733333333334,
  0.383644,
  0.28641,
  0.6432286666666667],
 'GaussianMixtureModels mean precision': [0.9935886070682411,
  0.9928387540435627,
  0.9965226514788198,
  0.9938809236880374,
  0.9965782598458853,
  0.9974958371615892,
  0.9875969617459274,
  0.9900191256368499,
  0.9946550076480944,
  0.9831255541587898,
  0.9939118286935877,
  0.9940490727739337,
  0.993222506488257,
  0.9965688874484172,
  0.9952805754642278,
  0.9951162615910039,
  0.9934165481340085,
  0.9949670870457992,
  0.9964660432274276,
  0.9959468275311019,
  0.9451014434562333,
  0.9472400640342195,
  0.9402581681900917,
  0.9408101950791198,
  0.9487884485735169,
  0.9410359328026497,
  0.9434298152142117,
  0.9447456414901977,
  0.9474408063396321,
  0.9497587266131877,
  0.9370616407658037,
  0.9328852354641952,
  0.9432587153346891,
  0.9419247518425979,
  0.9367971773878325,
  0.9362345788831263,
  0.935664390960946,
  0.9404365403642996,
  0.9442011624226416,
  0.9337176279832603,
  0.9751934811181174,
  0.9759996314292451,
  0.9748039683789954,
  0.9742125904455882,
  0.9757450800403673,
  0.9744635298163659,
  0.9747794304328813,
  0.9763874959024186,
  0.9749489933458917,
  0.9748357625965879,
  0.9903589875079256,
  0.9661112378284836,
  0.988464481535748,
  0.9755763542085504,
  0.9969054471394488,
  0.9836351733974316,
  0.9849988026051817,
  0.982888307095933,
  0.9829832211367694,
  0.9584380750612722],
 'GaussianMixtureModels mean coverage': [0.123265,
  0.12238333333333333,
  0.10837066666666667,
  0.11707466666666666,
  0.10728666666666667,
  0.12466233333333333,
  0.12601366666666666,
  0.11928733333333333,
  0.11101,
  0.141483,
  0.168937,
  0.15868733333333335,
  0.16740233333333335,
  0.15273566666666666,
  0.16031366666666666,
  0.15273466666666666,
  0.15848333333333334,
  0.170304,
  0.15643100000000001,
  0.143067,
  0.115373,
  0.10343966666666667,
  0.11871333333333334,
  0.112855,
  0.099447,
  0.119319,
  0.113914,
  0.10343366666666666,
  0.10519,
  0.10518033333333333,
  0.30507433333333334,
  0.30179066666666665,
  0.28794533333333333,
  0.29890666666666665,
  0.29692666666666667,
  0.3039,
  0.29527733333333334,
  0.2990123333333333,
  0.2866923333333333,
  0.3061333333333333,
  0.19163433333333332,
  0.20115266666666667,
  0.206933,
  0.20883433333333334,
  0.20554533333333333,
  0.202776,
  0.20165333333333332,
  0.19749,
  0.21164133333333335,
  0.20194566666666666,
  0.162045,
  0.18659666666666666,
  0.14361333333333334,
  0.14352366666666666,
  0.14329633333333333,
  0.14803033333333335,
  0.17669233333333334,
  0.14463766666666666,
  0.14985966666666667,
  0.20342066666666667]}


In [ ]:
len(data_dictionary["Data"])

In [ ]:
data = data_dictionary["Data"][dataset]

In [ ]:
import sklearn.ensemble

train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(data, labels, train_size=0.80)
blackbox_model = sklearn.ensemble.RandomForestClassifier(n_estimators=20)
blackbox_model.fit(train, labels_train)
print('Train', sklearn.metrics.accuracy_score(labels_train, blackbox_model.predict(train)))
print('Test', sklearn.metrics.accuracy_score(labels_test, blackbox_model.predict(test)))

In [ ]:
target_names = ['red', 'blue', 'green']
feature_names = ['x-axis', 'y-axis']

In [ ]:
explainer_1 = AnchorTabularExplainer(
    target_names,
    feature_names,
    train,
    train_label = labels_train,
    discretizer = 'quartile')

explainer_2 = AnchorTabularExplainer(
    target_names,
    feature_names,
    train,
    train_label = labels_train,
    discretizer = 'decile')

explainer_3 = AnchorTabularExplainer(
    target_names,
    feature_names,
    train,
    train_label = labels_train,
    discretizer = 'GaussianMixtureModels')

# GaussianMixtureModels
# quartile, decile

In [ ]:
idx = 150

np.random.seed(1)
print('Prediction: ', explainer_3.class_names[int(blackbox_model.predict(test[idx].reshape(1, -1))[0])])
""" cahnge exp = explainer.explain_instance(...) 
    to AnchorExplanation """
exp = explainer_3.explain_instance(test[idx], blackbox_model.predict, threshold=0.95)

In [ ]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())

print('Coverage: %.2f' % exp.coverage())

In [ ]:
import statistics
test_precision = []
test_coverage = []
for idx in range(len(test)):
    exp = explainer_1.explain_instance(test[idx], blackbox_model.predict, threshold=0.95)
    test_precision.append(exp.precision())
    test_coverage.append(exp.coverage())


print("mean precision", statistics.mean(test_precision))
print("mean coverage", statistics.mean(test_coverage))

results["quartile mean precision"].append(statistics.mean(test_precision))
results["quartile mean coverage"].append(statistics.mean(test_coverage))

In [ ]:
test_precision = []
test_coverage = []
for idx in range(len(test)):
    exp = explainer_2.explain_instance(test[idx], blackbox_model.predict, threshold=0.95)
    test_precision.append(exp.precision())
    test_coverage.append(exp.coverage())


print("mean precision", statistics.mean(test_precision))
print("mean coverage", statistics.mean(test_coverage))

results["decile mean precision"].append(statistics.mean(test_precision))
results["decile mean coverage"].append(statistics.mean(test_coverage))

In [ ]:
test_precision = []
test_coverage = []
print("Number of tests", len(test))
for idx in range(len(test)):
    if idx % 10 == 0 : print(idx)
    exp = explainer_3.explain_instance(test[idx], blackbox_model.predict, threshold=0.95)
    test_precision.append(exp.precision())
    test_coverage.append(exp.coverage())


print("mean precision", statistics.mean(test_precision))
print("mean coverage", statistics.mean(test_coverage))

results["GaussianMixtureModels mean precision"].append(statistics.mean(test_precision))
results["GaussianMixtureModels mean coverage"].append(statistics.mean(test_coverage))

In [ ]:
results["Number"].append(dataset)
results["Name"].append(data_dictionary["Name"][dataset])

In [ ]:
# GaussianMixtureModels
# quartile, decile

In [ ]:
results

In [1]:
# from lime.discretize import BaseDiscretizer
# class GaussianMixtureModelsDiscretizer(BaseDiscretizer):
    
#     def __init__(self, train, categorical_features, feature_names, labels, random_state=None, data_stats=None):
#         self.train = train
#         self.labels = labels
        
#         if(labels is None):
#             raise ValueError('Labels must be not None when using \
#                              GaussianMixtureModelsDiscretizer')
            
#         BaseDiscretizer.__init__(self, train, categorical_features,
#                                  feature_names, labels=labels,
#                                  random_state=random_state,
#                                  data_stats=data_stats)

    
#     def getLabels(self):
#         train_label = []
#         for lebel_number in set(self.labels):
#             Classification = np.array([self.train[i] for i in range(len(self.labels)) if self.labels[i] == lebel_number])
#             train_label.append(Classification)
        
#         return train_label
    
#     # BIC_range = make it an input??
#     def gmmBIC(self, train_label_list, BIC_range = [1, 10]):
    
#         all_gmm_list = []
#         for class_C in train_label_list:
#             gmm_list = []
#             BIC_list = []
#             for x in range(min(BIC_range), max(BIC_range) + 1):
#                 gmm = mixture.GaussianMixture(n_components= x, covariance_type='full').fit(class_C)
#                 BIC_list.append(gmm.bic(class_C))

#             min_BIC_index = BIC_list.index(min(BIC_list))
#             number_components = min_BIC_index + 1
#             gmm = mixture.GaussianMixture(n_components=number_components, covariance_type='full').fit(class_C)
#             gmm_list.append(gmm)

#             all_gmm_list.append(gmm_list[0])

#         return all_gmm_list
    
#     def getIntervalsGmm(self, gmm, std_constant=3):
    
#         width_interval_list = []
#         height_interval_list = []
#         pos_list = []
#         angle_list = []

#         # CHECK IF THIS WORKS FOR MORE THAN 2D, OTHERWISE CHANGE IT.
#         for pos, covariance, w in zip(gmm.means_, gmm.covariances_, gmm.weights_):

#             if covariance.shape == (2, 2):
#                 U, s, Vt = np.linalg.svd(covariance)
#                 angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
#                 width, height = std_constant * np.sqrt(s)
#             else:
#                 angle = 0
#                 width, height = std_constant * np.sqrt(covariance)



#             width_interval_list.append(width)
#             height_interval_list.append(height)
#             pos_list.append(pos)
#             angle_list.append(angle)

#         return pos_list, width_interval_list, height_interval_list, angle_list
    
    
#     def ellipseBoundingBox(self, pos_list, width_interval_list, height_interval_list, angle_list):
    
#         x_intervals = []
#         y_intervals = []
#         for n in range(len(pos_list)):

#             h = pos_list[n][0]
#             k = pos_list[n][1]
#             a = width_interval_list[n]
#             b = height_interval_list[n]
#             p = np.radians(angle_list[n])

#             x_1_angle = np.arctan(-b*np.tan(p)/a)
#             x_2_angle = np.arctan(-b*np.tan(p)/a) + math.pi

#             y_1_angle = np.arctan(b/(a*np.tan(p)))
#             y_2_angle = np.arctan(b/(a*np.tan(p))) + math.pi

#             x_1 = h + a*np.cos(x_1_angle)*np.cos(p) - b*np.sin(x_1_angle)*np.sin(p)
#             x_2 = h + a*np.cos(x_2_angle)*np.cos(p) - b*np.sin(x_2_angle)*np.sin(p)

#             y_1 = k + b*np.sin(y_1_angle)*np.cos(p) + a*np.cos(y_1_angle)*np.sin(p)
#             y_2 = k + b*np.sin(y_2_angle)*np.cos(p) + a*np.cos(y_2_angle)*np.sin(p)

#             x_intervals.append([x_1, x_2])
#             y_intervals.append([y_1, y_2])

#         return x_intervals, y_intervals
     
    
#     # make this for more than 2-d
#     def getBoundingBoxesIntervals(self, list_gmm):
    
#         x_intervals_list = []
#         y_intervals_list = []
#         for gmm in list_gmm:
#             pos_list, width_interval_list, height_interval_list, angle_list = self.getIntervalsGmm(gmm)
#             x_intervals, y_intervals = self.ellipseBoundingBox(pos_list, width_interval_list, height_interval_list, angle_list)
            
#             x_flat_intervals = [interval for intervals in x_intervals for interval in intervals]
#             y_flat_intervals = [interval for intervals in y_intervals for interval in intervals]
#             x_intervals_list.append(x_flat_intervals)
#             y_intervals_list.append(y_flat_intervals)
            
        
#         x_flat_intervals_list = [interval for intervals in x_intervals_list for interval in intervals]
#         y_flat_intervals_list = [interval for intervals in y_intervals_list for interval in intervals]
#         x_flat_intervals_list.sort()
#         y_flat_intervals_list.sort()

#         feature_intervals_lists = [np.array(x_flat_intervals_list), np.array(y_flat_intervals_list)]
        
#         return feature_intervals_lists

    
       
        
#     def bins(self, data, labels):
        
#         train_label = self.getLabels()
#         all_gmm_list = self.gmmBIC(train_label)
#         feature_intervals_lists = self.getBoundingBoxesIntervals(all_gmm_list)
        
#         bins = feature_intervals_lists
        
#         return bins
    
    

In [ ]:
results = {"Number" : [], "Name": [], "quartile mean precision": [], "quartile mean coverage": [], "decile mean precision": [], "decile mean coverage": [], "entropy mean precision": [], "entropy mean coverage": [], "GaussianMixtureModels mean precision": [], "GaussianMixtureModels mean coverage": []}